In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
from skimage import io, filters, exposure, restoration, img_as_float, img_as_uint
import glob
import tifffile
from pathlib import Path
import matplotlib.pyplot as plt

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/trial'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def normalize_image(img):
    """Normalize image to 0-1 range"""
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:
        return (img - img_min) / (img_max - img_min)
    return img

def focus_stack_projection(image_stack):
    """
    Create a projection using focus measure (variance of Laplacian)

    Parameters:
    -----------
    image_stack : numpy.ndarray
        Stack of images with shape (z, y, x)

    Returns:
    --------
    numpy.ndarray
        Focus-stacked image
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Calculate focus measure for each pixel in each slice
    focus_measures = np.zeros_like(stack)
    for i in range(stack.shape[0]):
        # Use variance of Laplacian as focus measure
        lap = filters.laplace(stack[i])
        focus_measures[i] = filters.gaussian(np.abs(lap), sigma=1)

    # For each pixel position, find the z-slice with maximum focus measure
    indices = np.argmax(focus_measures, axis=0)

    # Create empty output image
    output = np.zeros_like(stack[0])

    # Fill output image with pixels from the z-slice with best focus
    for i in range(stack.shape[0]):
        mask = (indices == i)
        output[mask] = stack[i][mask]

    return output

def entropy_projection(image_stack, window_size=7):
    """
    Create a projection using local entropy as focus measure

    Parameters:
    -----------
    image_stack : numpy.ndarray
        Stack of images with shape (z, y, x)
    window_size : int
        Size of the window for entropy calculation

    Returns:
    --------
    numpy.ndarray
        Entropy-based projected image
    """
    from skimage.filters.rank import entropy
    from skimage.morphology import disk

    # Convert to float and normalize
    stack = img_as_float(image_stack)

    # Calculate entropy for each slice
    entropy_measures = np.zeros_like(stack)
    selem = disk(window_size // 2)

    for i in range(stack.shape[0]):
        # Convert to uint8 for entropy calculation
        uint8_img = (normalize_image(stack[i]) * 255).astype(np.uint8)
        entropy_measures[i] = entropy(uint8_img, selem)

    # For each pixel position, find the z-slice with maximum entropy
    indices = np.argmax(entropy_measures, axis=0)

    # Create empty output image
    output = np.zeros_like(stack[0])

    # Fill output image with pixels from the z-slice with maximum entropy
    for i in range(stack.shape[0]):
        mask = (indices == i)
        output[mask] = stack[i][mask]

    return output

def contrast_weighted_projection(image_stack):
    """
    Create a weighted average projection where weights are proportional to local contrast

    Parameters:
    -----------
    image_stack : numpy.ndarray
        Stack of images with shape (z, y, x)

    Returns:
    --------
    numpy.ndarray
        Contrast-weighted projected image
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Calculate weights based on local contrast for each slice
    weights = np.zeros_like(stack)
    for i in range(stack.shape[0]):
        # Use variance of Laplacian for local contrast
        lap = filters.laplace(stack[i])
        weights[i] = filters.gaussian(np.abs(lap), sigma=2)

    # Normalize weights for each pixel position
    weight_sum = np.sum(weights, axis=0)
    weight_sum[weight_sum == 0] = 1  # Avoid division by zero

    # Initialize output with zeros
    output = np.zeros_like(stack[0])

    # Calculate weighted average
    for i in range(stack.shape[0]):
        output += stack[i] * weights[i] / weight_sum

    return output

def max_intensity_projection(image_stack):
    """Simple maximum intensity projection"""
    return np.max(image_stack, axis=0)

def std_projection(image_stack):
    """Standard deviation projection (useful to highlight variance)"""
    return np.std(image_stack, axis=0)

def process_4d_microscopy_image(image, filename):
    """
    Process a 4D microscopy image with shape (channels, z-slices, height, width)
    Apply different projection methods

    Parameters:
    -----------
    image : numpy.ndarray
        4D input image with shape (channels, z-slices, height, width)
    filename : str
        Original filename for naming the output

    Returns:
    --------
    dict
        Dictionary of processed images
    """
    print(f"  Processing 4D image with shape {image.shape}")

    # Normalize values if needed
    if image.dtype == np.float32 or image.dtype == np.float64:
        # Check if values are outside expected range
        if np.max(image) > 1.0 or np.min(image) < -1.0:
            print(f"  Normalizing image values from range [{np.min(image)}, {np.max(image)}] to [0, 1]")
            for c in range(image.shape[0]):
                for z in range(image.shape[1]):
                    image[c, z] = normalize_image(image[c, z])

    projections = {}

    # Process each channel
    for channel in range(image.shape[0]):
        # Get all z-slices for this channel
        channel_data = image[channel]

        # Apply different projection methods
        projection_methods = {
            'max': max_intensity_projection,
            'focus': focus_stack_projection,
            'entropy': entropy_projection,
            'contrast': contrast_weighted_projection,
            'std': std_projection
        }

        # Apply each method and enhance result
        for method_name, method_func in projection_methods.items():
            try:
                # Apply projection method
                proj = method_func(channel_data)

                # Enhance contrast with adaptive histogram equalization
                enhanced = exposure.equalize_adapthist(proj, clip_limit=0.03)

                # Add sharpening only for non-focus methods (focus already has edge enhancement)
                if method_name != 'focus' and method_name != 'entropy':
                    enhanced = filters.unsharp_mask(enhanced, radius=2, amount=1.5)

                # Store result
                key = f"channel_{channel+1}_{method_name}"
                projections[key] = enhanced
            except Exception as e:
                print(f"  Error applying {method_name} projection: {e}")

    return projections

def process_all_images():
    """
    Process all 4D microscopy images in the input directory
    """
    # Get all image files with common extensions (case insensitive)
    extensions = ['.tif', '.tiff', '.TIF', '.TIFF']
    all_files = []

    for ext in extensions:
        all_files.extend(glob.glob(os.path.join(input_dir, f'*{ext}')))

    if len(all_files) == 0:
        print(f"No image files found in {input_dir}")
        return []

    # Sort files to ensure consistent processing order
    all_files.sort()

    print(f"Found {len(all_files)} image files. Processing each with multiple methods...")

    processed_files = []

    # Process each image
    for idx, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        print(f"Processing image {idx+1}/{len(all_files)}: {filename}")

        try:
            # Load the image
            image = tifffile.imread(file_path)

            # Print image info
            print(f"  Image shape: {image.shape}, dtype: {image.dtype}")

            # Skip if not 4D
            if len(image.shape) != 4:
                print(f"  Skipping - not a 4D image")
                continue

            # Process the 4D image
            projections = process_4d_microscopy_image(image, filename)

            # Save each projection
            for proj_name, projection in projections.items():
                # Create output filename
                base_name = os.path.splitext(filename)[0]
                output_filename = f"{base_name}_{proj_name}.tif"
                output_path = os.path.join(output_dir, output_filename)

                # Convert to uint16 for saving
                if projection.dtype == np.float64 or projection.dtype == np.float32:
                    projection = img_as_uint(projection)

                # Save the projection
                tifffile.imwrite(output_path, projection)

                # Add to list of processed files
                processed_files.append(output_path)

            print(f"  Saved {len(projections)} projections")

        except Exception as e:
            print(f"  Error processing image {filename}: {e}")
            import traceback
            traceback.print_exc()
            continue

    return processed_files

# Main execution
print("Starting 4D image processing with multiple projection methods...")

# Process all image files in the directory
processed_files = process_all_images()

# Display results
if processed_files:
    print(f"\nSuccessfully processed {len(processed_files)} projections:")
    for output_file in processed_files[:5]:  # Show first 5
        print(f"  - {os.path.basename(output_file)}")
    if len(processed_files) > 5:
        print(f"  ... and {len(processed_files) - 5} more")

    # Try to display the first result in the notebook
    try:
        from IPython.display import Image
        print("\nDisplaying first result:")
        Image(processed_files[0])
    except Exception as e:
        print(f"Couldn't display image: {e}")
else:
    print("No images were processed successfully.")

Mounted at /content/drive
Starting 4D image processing with multiple projection methods...
Found 1 image files. Processing each with multiple methods...
Processing image 1/1: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1).tif
  Image shape: (3, 21, 1024, 1024), dtype: float32
  Processing 4D image with shape (3, 21, 1024, 1024)
  Normalizing image values from range [-0.02250508777797222, 3.4323275089263916] to [0, 1]
  Saved 15 projections

Successfully processed 15 projections:
  - denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1)_channel_1_max.tif
  - denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1)_channel_1_focus.tif
  - denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1)_channel_1_entropy.tif
  - denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1)_channel_1_contrast.tif
  - denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1)_channel_1_std.tif
  ... and 10 more

Displaying first result:
Couldn't display image: Cannot embed the 'tif' image format


In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
from skimage import io, filters, exposure, restoration, img_as_float, img_as_uint
import glob
import tifffile
from pathlib import Path
import matplotlib.pyplot as plt

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/trial2'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def normalize_image(img):
    """Normalize image to 0-1 range"""
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:
        return (img - img_min) / (img_max - img_min)
    return img

# Original projection methods
def max_intensity_projection(image_stack):
    """Simple maximum intensity projection"""
    return np.max(image_stack, axis=0)

def focus_stack_projection(image_stack):
    """
    Create a projection using focus measure (variance of Laplacian)
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Calculate focus measure for each pixel in each slice
    focus_measures = np.zeros_like(stack)
    for i in range(stack.shape[0]):
        # Use variance of Laplacian as focus measure
        lap = filters.laplace(stack[i])
        focus_measures[i] = filters.gaussian(np.abs(lap), sigma=1)

    # For each pixel position, find the z-slice with maximum focus measure
    indices = np.argmax(focus_measures, axis=0)

    # Create empty output image
    output = np.zeros_like(stack[0])

    # Fill output image with pixels from the z-slice with best focus
    for i in range(stack.shape[0]):
        mask = (indices == i)
        output[mask] = stack[i][mask]

    return output

def entropy_projection(image_stack, window_size=7):
    """
    Create a projection using local entropy as focus measure
    """
    from skimage.filters.rank import entropy
    from skimage.morphology import disk

    # Convert to float and normalize
    stack = img_as_float(image_stack)

    # Calculate entropy for each slice
    entropy_measures = np.zeros_like(stack)
    selem = disk(window_size // 2)

    for i in range(stack.shape[0]):
        # Convert to uint8 for entropy calculation
        uint8_img = (normalize_image(stack[i]) * 255).astype(np.uint8)
        entropy_measures[i] = entropy(uint8_img, selem)

    # For each pixel position, find the z-slice with maximum entropy
    indices = np.argmax(entropy_measures, axis=0)

    # Create empty output image
    output = np.zeros_like(stack[0])

    # Fill output image with pixels from the z-slice with maximum entropy
    for i in range(stack.shape[0]):
        mask = (indices == i)
        output[mask] = stack[i][mask]

    return output

def contrast_weighted_projection(image_stack):
    """
    Create a weighted average projection where weights are proportional to local contrast
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Calculate weights based on local contrast for each slice
    weights = np.zeros_like(stack)
    for i in range(stack.shape[0]):
        # Use variance of Laplacian for local contrast
        lap = filters.laplace(stack[i])
        weights[i] = filters.gaussian(np.abs(lap), sigma=2)

    # Normalize weights for each pixel position
    weight_sum = np.sum(weights, axis=0)
    weight_sum[weight_sum == 0] = 1  # Avoid division by zero

    # Initialize output with zeros
    output = np.zeros_like(stack[0])

    # Calculate weighted average
    for i in range(stack.shape[0]):
        output += stack[i] * weights[i] / weight_sum

    return output

def std_projection(image_stack):
    """Standard deviation projection (useful to highlight variance)"""
    return np.std(image_stack, axis=0)

# New projection methods
def min_intensity_projection(image_stack):
    """Minimum intensity projection - useful for visualizing air-filled structures"""
    return np.min(image_stack, axis=0)

def average_intensity_projection(image_stack):
    """Average intensity projection - provides a balanced view of structures"""
    return np.mean(image_stack, axis=0)

def surface_projection(image_stack, threshold=0.1):
    """
    Surface projection - represents the first structure encountered above a threshold
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create empty output image
    output = np.zeros_like(stack[0])

    # For each pixel position, find the first z-slice that exceeds the threshold
    for y in range(stack.shape[1]):
        for x in range(stack.shape[2]):
            for z in range(stack.shape[0]):
                if stack[z, y, x] > threshold:
                    output[y, x] = stack[z, y, x]
                    break

    return output

def depth_coded_projection(image_stack):
    """
    Depth-coded projection - uses color to encode depth information
    Since we can only return a grayscale image, we encode depth as intensity
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Find the maximum value at each position and its index
    max_vals = np.max(stack, axis=0)
    max_indices = np.argmax(stack, axis=0)

    # Normalize the depth values to 0-1 range
    depth_map = max_indices.astype(float) / (stack.shape[0] - 1)

    # Combine intensity and depth (weighted combination)
    # Here we give 70% weight to intensity and 30% to depth
    output = 0.7 * max_vals + 0.3 * depth_map

    return output

def local_intensity_projection(image_stack, neighborhood=3):
    """
    Local intensity projection - projects local intensity extrema
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Calculate local maxima for each slice
    local_maxima = np.zeros_like(stack)
    for i in range(stack.shape[0]):
        # Find local maxima using maximum filter
        from scipy.ndimage import maximum_filter
        local_maxima[i] = maximum_filter(stack[i], size=neighborhood)

    # For each pixel position, find the z-slice with maximum local maximum
    indices = np.argmax(local_maxima, axis=0)

    # Create empty output image
    output = np.zeros_like(stack[0])

    # Fill output image with pixels from the selected z-slices
    for i in range(stack.shape[0]):
        mask = (indices == i)
        output[mask] = stack[i][mask]

    return output

def weighted_average_projection(image_stack, sigma=1.0):
    """
    Weighted average projection - similar to AIP but with depth-dependent weighting
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create weights that emphasize the center of the stack
    z_positions = np.arange(stack.shape[0])
    center = stack.shape[0] / 2
    weights = np.exp(-((z_positions - center) ** 2) / (2 * sigma ** 2))

    # Reshape weights for broadcasting
    weights = weights.reshape(-1, 1, 1)

    # Calculate weighted sum and normalize
    weighted_sum = np.sum(stack * weights, axis=0)
    weight_sum = np.sum(weights)

    return weighted_sum / weight_sum

def tensor_projection(image_stack):
    """
    Simplified tensor projection - creates a directional emphasis
    For full tensor projection, diffusion tensor data would be needed
    """
    # Calculate gradients in z direction
    gradients = np.zeros_like(image_stack)
    for i in range(1, image_stack.shape[0]):
        gradients[i] = image_stack[i] - image_stack[i-1]

    # Take the absolute value of gradients
    abs_gradients = np.abs(gradients)

    # Weight the original stack by the gradient magnitude
    weights = filters.gaussian(abs_gradients, sigma=1)

    # Normalize weights
    weight_sum = np.sum(weights, axis=0)
    weight_sum[weight_sum == 0] = 1  # Avoid division by zero

    # Create output image
    output = np.zeros_like(image_stack[0])

    # Calculate weighted projection
    for i in range(image_stack.shape[0]):
        output += image_stack[i] * weights[i] / weight_sum

    return output

def process_4d_microscopy_image(image, filename, process_first_channel_only=True):
    """
    Process a 4D microscopy image with shape (channels, z-slices, height, width)
    Apply different projection methods

    Parameters:
    -----------
    image : numpy.ndarray
        4D input image with shape (channels, z-slices, height, width)
    filename : str
        Original filename for naming the output
    process_first_channel_only : bool
        If True, only process the first channel

    Returns:
    --------
    dict
        Dictionary of processed images
    """
    print(f"  Processing 4D image with shape {image.shape}")

    # Normalize values if needed
    if image.dtype == np.float32 or image.dtype == np.float64:
        # Check if values are outside expected range
        if np.max(image) > 1.0 or np.min(image) < -1.0:
            print(f"  Normalizing image values from range [{np.min(image)}, {np.max(image)}] to [0, 1]")
            for c in range(image.shape[0]):
                for z in range(image.shape[1]):
                    image[c, z] = normalize_image(image[c, z])

    projections = {}

    # Determine which channels to process
    channels_to_process = [0] if process_first_channel_only else range(image.shape[0])

    # Process each channel
    for channel in channels_to_process:
        # Get all z-slices for this channel
        channel_data = image[channel]

        # Define all projection methods
        projection_methods = {
            # Original methods
            'max': max_intensity_projection,
            'focus': focus_stack_projection,
            'entropy': entropy_projection,
            'contrast': contrast_weighted_projection,
            'std': std_projection,

            # New methods
            'min': min_intensity_projection,
            'avg': average_intensity_projection,
            'surface': surface_projection,
            'depth': depth_coded_projection,
            'local': local_intensity_projection,
            'weighted_avg': weighted_average_projection,
            'tensor': tensor_projection
        }

        # Apply each method and enhance result
        for method_name, method_func in projection_methods.items():
            try:
                # Apply projection method
                print(f"  Applying {method_name} projection to channel {channel+1}...")
                proj = method_func(channel_data)

                # Enhance contrast with adaptive histogram equalization
                enhanced = exposure.equalize_adapthist(proj, clip_limit=0.03)

                # Add sharpening only for certain methods
                if method_name not in ['focus', 'entropy', 'local', 'tensor']:
                    enhanced = filters.unsharp_mask(enhanced, radius=2, amount=1.5)

                # Store result
                key = f"channel_{channel+1}_{method_name}"
                projections[key] = enhanced
                print(f"  Successfully created {key} projection")
            except Exception as e:
                print(f"  Error applying {method_name} projection: {e}")
                import traceback
                traceback.print_exc()

    return projections

def process_all_images():
    """
    Process all 4D microscopy images in the input directory
    """
    # Get all image files with common extensions (case insensitive)
    extensions = ['.tif', '.tiff', '.TIF', '.TIFF']
    all_files = []

    for ext in extensions:
        all_files.extend(glob.glob(os.path.join(input_dir, f'*{ext}')))

    if len(all_files) == 0:
        print(f"No image files found in {input_dir}")
        return []

    # Sort files to ensure consistent processing order
    all_files.sort()

    print(f"Found {len(all_files)} image files. Processing each with multiple methods...")

    processed_files = []

    # Process each image
    for idx, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        print(f"Processing image {idx+1}/{len(all_files)}: {filename}")

        try:
            # Load the image
            image = tifffile.imread(file_path)

            # Print image info
            print(f"  Image shape: {image.shape}, dtype: {image.dtype}")

            # Skip if not 4D
            if len(image.shape) != 4:
                print(f"  Skipping - not a 4D image")
                continue

            # Process the 4D image (only first channel)
            projections = process_4d_microscopy_image(image, filename, process_first_channel_only=True)

            # Save each projection
            for proj_name, projection in projections.items():
                # Create output filename
                base_name = os.path.splitext(filename)[0]
                output_filename = f"{base_name}_{proj_name}.tif"
                output_path = os.path.join(output_dir, output_filename)

                # Convert to uint16 for saving
                if projection.dtype == np.float64 or projection.dtype == np.float32:
                    projection = img_as_uint(projection)

                # Save the projection
                tifffile.imwrite(output_path, projection)

                # Add to list of processed files
                processed_files.append(output_path)

            print(f"  Saved {len(projections)} projections")

        except Exception as e:
            print(f"  Error processing image {filename}: {e}")
            import traceback
            traceback.print_exc()
            continue

    return processed_files

# Main execution
print("Starting 4D image processing with enhanced projection methods...")

# Process all image files in the directory
processed_files = process_all_images()

# Display results
if processed_files:
    print(f"\nSuccessfully processed {len(processed_files)} projections:")
    for output_file in processed_files[:5]:  # Show first 5
        print(f"  - {os.path.basename(output_file)}")
    if len(processed_files) > 5:
        print(f"  ... and {len(processed_files) - 5} more")

    # Try to display the first result in the notebook
    try:
        from IPython.display import Image
        print("\nDisplaying first result:")
        Image(processed_files[0])
    except Exception as e:
        print(f"Couldn't display image: {e}")
else:
    print("No images were processed successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting 4D image processing with enhanced projection methods...
Found 1 image files. Processing each with multiple methods...
Processing image 1/1: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1).tif
  Image shape: (3, 21, 1024, 1024), dtype: float32
  Processing 4D image with shape (3, 21, 1024, 1024)
  Normalizing image values from range [-0.02250508777797222, 3.4323275089263916] to [0, 1]
  Applying max projection to channel 1...
  Successfully created channel_1_max projection
  Applying focus projection to channel 1...
  Successfully created channel_1_focus projection
  Applying entropy projection to channel 1...
  Successfully created channel_1_entropy projection
  Applying contrast projection to channel 1...
  Successfully created channel_1_contrast projection
  Applying std projection to channel 1...
  Successfully created channel_1_std projection

In [3]:
import numpy as np
from skimage import filters, morphology, feature, measure, exposure, segmentation, img_as_float, img_as_uint
from scipy import ndimage
import matplotlib.pyplot as plt

def normalize_image(img):
    """Normalize image to 0-1 range"""
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:
        return (img - img_min) / (img_max - img_min)
    return img

# Advanced projections specifically for detecting black holes/vacuoles in cadherin-stained endothelial cells

def inverse_weighted_projection(image_stack, sigma=1.5):
    """
    Inverse-weighted projection that emphasizes dark regions (potential vacuoles)
    by giving higher weight to darker voxels in the z-direction
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Invert the stack to emphasize dark regions
    inverted_stack = 1 - stack

    # Calculate weights based on darkness (higher value = darker pixel)
    weights = filters.gaussian(inverted_stack, sigma=sigma)

    # Normalize weights for each pixel position
    weight_sum = np.sum(weights, axis=0)
    weight_sum[weight_sum == 0] = 1  # Avoid division by zero

    # Calculate weighted average of original (non-inverted) stack
    output = np.zeros_like(stack[0])
    for i in range(stack.shape[0]):
        output += stack[i] * weights[i] / weight_sum

    return output

def dark_spot_enhancement_projection(image_stack, spot_size=5):
    """
    Projects the stack while enhancing dark spots of a specific size range
    Useful for highlighting vacuoles/holes in cadherin-stained cells
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create output with maximum values first
    output = np.max(stack, axis=0)

    # Process each z-slice to detect dark spots
    for z in range(stack.shape[0]):
        # Detect dark spots using white top-hat transform on inverted image
        inverted = 1 - stack[z]
        selem = morphology.disk(spot_size)
        dark_spots = morphology.white_tophat(inverted, selem)

        # Threshold to identify significant dark spots
        threshold = filters.threshold_otsu(dark_spots) * 0.5
        dark_spots_binary = dark_spots > threshold

        # Label connected regions
        labeled_spots = measure.label(dark_spots_binary)

        # For each labeled region, check if it's a potential vacuole
        for region in measure.regionprops(labeled_spots):
            # Filter based on size and shape
            if region.area > 10 and region.area < 500 and region.eccentricity < 0.8:
                # Get region coordinates
                coords = region.coords

                # Update output image to show this dark spot
                for coord in coords:
                    y, x = coord
                    # Replace with dark value from this slice if it's darker
                    if stack[z, y, x] < output[y, x]:
                        output[y, x] = stack[z, y, x]

    return output

def junction_gap_projection(image_stack, membrane_thickness=3):
    """
    Specifically designed to detect gaps in cadherin staining at cell junctions
    by looking for discontinuities in membrane staining patterns
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Get maximum intensity projection as base
    mip = np.max(stack, axis=0)

    # Enhance edges to highlight cell boundaries
    edges = np.zeros_like(mip)
    for z in range(stack.shape[0]):
        # Detect edges using Sobel filter
        edge_sobel = filters.sobel(stack[z])
        edges = np.maximum(edges, edge_sobel)

    # Threshold edges to get membrane mask
    edge_threshold = filters.threshold_otsu(edges) * 0.5
    membrane_mask = edges > edge_threshold

    # Dilate to connect nearby membrane segments
    membrane_mask = morphology.binary_dilation(membrane_mask, morphology.disk(membrane_thickness))

    # Find potential gaps - areas where membrane should be but intensity is low
    gap_candidates = np.zeros_like(mip)

    # Process each z-slice to look for gaps in membranes
    for z in range(stack.shape[0]):
        # Get regions where membrane mask exists but pixel intensity is low
        low_intensity = stack[z] < np.percentile(stack[z][membrane_mask], 30)
        potential_gaps = membrane_mask & low_intensity

        # Add to candidates
        gap_candidates[potential_gaps] += 1

    # Normalize gap candidates
    gap_candidates = gap_candidates / stack.shape[0]

    # Create output that emphasizes gaps in junctions
    output = mip.copy()

    # Highlight gap regions by reducing intensity
    gap_regions = gap_candidates > 0.5
    output[gap_regions] = output[gap_regions] * 0.5

    return output

def hollow_structure_projection(image_stack, threshold=0.2, min_size=20, max_size=500):
    """
    Projects 3D hollow structures (potential vacuoles) by identifying dark regions
    surrounded by brighter regions across multiple z-slices
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Base projection - maximum intensity
    output = np.max(stack, axis=0)

    # Store potential hollow structures
    hollow_mask = np.zeros_like(output, dtype=bool)

    # Process each z slice
    for z in range(1, stack.shape[0]-1):  # Skip first and last slice
        # Threshold this slice to get foreground
        binary = stack[z] > threshold

        # Find holes in the binary image
        holes = morphology.remove_small_holes(binary, min_size) != binary

        # Remove small objects
        holes = morphology.remove_small_objects(holes, min_size)

        # Label connected components
        labeled_holes = measure.label(holes)

        # For each hole, check if it's surrounded by higher intensity in z-direction
        for region in measure.regionprops(labeled_holes):
            if region.area < max_size:  # Size constraint
                # Get region mask
                y0, x0, y1, x1 = region.bbox
                region_mask = labeled_holes[y0:y1, x0:x1] == region.label

                # Check intensity above and below
                above_intensity = np.mean(stack[min(z+1, stack.shape[0]-1), y0:y1, x0:x1][region_mask])
                below_intensity = np.mean(stack[max(z-1, 0), y0:y1, x0:x1][region_mask])
                current_intensity = np.mean(stack[z, y0:y1, x0:x1][region_mask])

                # If current slice is darker than slices above and below, it might be a hollow structure
                if current_intensity < above_intensity*0.8 and current_intensity < below_intensity*0.8:
                    hollow_mask[y0:y1, x0:x1] |= region_mask

    # Enhance the visualization by making hollow regions darker
    output[hollow_mask] = output[hollow_mask] * 0.3

    return output

def differential_density_projection(image_stack, kernel_size=5):
    """
    Highlights regions where there's significant difference in density/intensity
    between adjacent z-layers, useful for detecting vacuole boundaries
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Calculate gradient magnitude in z direction
    z_gradient = np.zeros_like(stack)
    for z in range(1, stack.shape[0]):
        z_gradient[z] = np.abs(stack[z] - stack[z-1])

    # Smooth the gradient
    smoothed_gradient = np.zeros_like(z_gradient)
    for z in range(z_gradient.shape[0]):
        smoothed_gradient[z] = filters.gaussian(z_gradient[z], sigma=1)

    # Get the maximum gradient projection
    max_gradient = np.max(smoothed_gradient, axis=0)

    # Create output that combines intensity and gradient information
    # Higher gradient indicates boundary of structure
    output = np.max(stack, axis=0)

    # Enhance regions with high gradient
    norm_gradient = normalize_image(max_gradient)
    output = output * (1 - norm_gradient * 0.5)

    return output

def phase_contrast_simulation_projection(image_stack, sigma=2):
    """
    Simulates phase contrast-like visualization to enhance membrane and vacuole boundaries
    by emphasizing intensity gradients in all directions
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create output image
    output = np.zeros_like(stack[0])

    # Process each z slice
    for z in range(stack.shape[0]):
        # Apply Laplacian of Gaussian filter to simulate phase contrast
        log_filtered = filters.laplace(filters.gaussian(stack[z], sigma=sigma))

        # Normalize to -1 to 1 range
        log_filtered = 2 * normalize_image(log_filtered) - 1

        # Convert to phase contrast-like appearance
        phase_contrast = 0.5 + 0.5 * log_filtered

        # Add to output based on original intensity (weighted)
        output += phase_contrast * stack[z]

    # Normalize
    output = normalize_image(output)

    return output

def local_minima_projection(image_stack, footprint_size=5):
    """
    Projects local minima across the z-stack, which could correspond to vacuoles
    or gaps in cadherin staining
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize with maximum values
    output = np.ones_like(stack[0])

    # Process each z slice
    for z in range(stack.shape[0]):
        # Find local minima
        footprint = morphology.disk(footprint_size)
        local_min = feature.peak_local_max(
            -stack[z],  # Invert to find minima
            footprint=footprint,
            indices=False
        )

        # For each local minimum, check if it's a potential vacuole
        labeled_mins = measure.label(local_min)

        for region in measure.regionprops(labeled_mins):
            y, x = region.centroid
            y, x = int(y), int(x)

            # Update output with local minimum value if it's lower
            if stack[z, y, x] < output[y, x]:
                # Update a small region around the minimum
                y_min, x_min = max(0, y-2), max(0, x-2)
                y_max, x_max = min(stack.shape[1], y+3), min(stack.shape[2], x+3)

                # Check each pixel in the neighborhood
                for yi in range(y_min, y_max):
                    for xi in range(x_min, x_max):
                        if stack[z, yi, xi] < output[yi, xi]:
                            output[yi, xi] = stack[z, yi, xi]

    return output

def texture_difference_projection(image_stack, window_size=9):
    """
    Emphasizes areas where texture differs significantly from surroundings,
    which may indicate vacuoles in cadherin-stained cell membranes
    """
    from skimage.feature import graycomatrix, graycoprops

    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Get maximum intensity projection as base
    output = np.max(stack, axis=0)

    # Texture difference mask
    texture_diff = np.zeros_like(output)

    # Process each z slice
    for z in range(stack.shape[0]):
        # Quantize image for GLCM calculation
        img_quantized = np.round(stack[z] * 31).astype(np.uint8)

        # Calculate local texture properties using sliding window
        for y in range(0, stack.shape[1]-window_size, window_size//2):
            for x in range(0, stack.shape[2]-window_size, window_size//2):
                # Extract patch
                patch = img_quantized[y:y+window_size, x:x+window_size]

                # Skip if patch is too small
                if patch.shape[0] < 3 or patch.shape[1] < 3:
                    continue

                # Calculate GLCM
                glcm = graycomatrix(patch, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=32, symmetric=True, normed=True)

                # Calculate texture properties
                contrast = graycoprops(glcm, 'contrast')[0].mean()
                homogeneity = graycoprops(glcm, 'homogeneity')[0].mean()
                energy = graycoprops(glcm, 'energy')[0].mean()

                # Identify regions with high contrast and low homogeneity (potential boundaries)
                # or very high homogeneity (potential vacuoles)
                if (contrast > 0.5 and homogeneity < 0.5) or homogeneity > 0.95:
                    texture_diff[y:y+window_size, x:x+window_size] += 1

    # Normalize texture difference
    texture_diff = texture_diff / stack.shape[0]

    # Enhance output based on texture differences
    # Make regions with high texture difference more prominent
    texture_mask = texture_diff > 0.5
    output[texture_mask] = output[texture_mask] * 0.7

    return output

def vesicle_enhancing_projection(image_stack, min_size=5, max_size=50):
    """
    Specifically designed to enhance small vesicles or vacuoles
    by detecting spherical hollow structures across z-slices
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize outputs
    output = np.max(stack, axis=0)  # Base is maximum projection
    vesicle_mask = np.zeros_like(output, dtype=bool)

    # Step 1: Detect circular structures in each slice
    for z in range(stack.shape[0]):
        # Apply a white tophat filter to enhance small structures
        tophat = morphology.white_tophat(1-stack[z], morphology.disk(max_size//2))

        # Threshold to get potential vesicles
        thresh = filters.threshold_otsu(tophat) * 0.7
        binary = tophat > thresh

        # Remove small noise and fill holes
        binary = morphology.remove_small_objects(binary, min_size)
        binary = morphology.remove_small_holes(binary)

        # Label connected components
        labeled = measure.label(binary)

        # Analyze each region
        for region in measure.regionprops(labeled):
            # Filter based on size and circularity
            if (min_size < region.area < max_size and
                region.eccentricity < 0.6 and  # Circular shape
                region.solidity > 0.8):        # Solid/compact

                # Get coordinates
                y0, x0, y1, x1 = region.bbox
                mask = labeled[y0:y1, x0:x1] == region.label

                # Check intensity inside vs outside
                inside_intensity = np.mean(stack[z, y0:y1, x0:x1][mask])

                # Create dilated mask for outside region
                dilated_mask = morphology.binary_dilation(mask, morphology.disk(2))
                dilated_mask = dilated_mask & ~mask

                # Check if there's enough border pixels
                if np.sum(dilated_mask) > 0:
                    outside_intensity = np.mean(stack[z, y0:y1, x0:x1][dilated_mask])

                    # If inside is significantly darker than outside, it's a potential vesicle
                    if inside_intensity < outside_intensity * 0.7:
                        vesicle_mask[y0:y1, x0:x1] |= mask

    # Enhance visualization by darkening detected vesicles
    output[vesicle_mask] = output[vesicle_mask] * 0.2

    # And brightening their boundaries for better visibility
    vesicle_boundary = morphology.binary_dilation(vesicle_mask) & ~vesicle_mask
    output[vesicle_boundary] = np.minimum(output[vesicle_boundary] * 1.5, 1.0)

    return output

# Example of how to integrate these new projections into your existing code

def process_4d_microscopy_image(image, filename, process_first_channel_only=True):
    """
    Process a 4D microscopy image with advanced projections for vacuole detection

    Parameters:
    -----------
    image : numpy.ndarray
        4D input image with shape (channels, z-slices, height, width)
    filename : str
        Original filename for naming the output
    process_first_channel_only : bool
        If True, only process the first channel

    Returns:
    --------
    dict
        Dictionary of processed images
    """
    print(f"  Processing 4D image with shape {image.shape}")

    # Normalize values if needed
    if image.dtype == np.float32 or image.dtype == np.float64:
        if np.max(image) > 1.0 or np.min(image) < -1.0:
            print(f"  Normalizing image values from range [{np.min(image)}, {np.max(image)}] to [0, 1]")
            for c in range(image.shape[0]):
                for z in range(image.shape[1]):
                    image[c, z] = normalize_image(image[c, z])

    projections = {}

    # Determine which channels to process
    channels_to_process = [0] if process_first_channel_only else range(image.shape[0])

    # Process each channel
    for channel in channels_to_process:
        # Get all z-slices for this channel
        channel_data = image[channel]

        # Define vacuole/gap detection projection methods
        vacuole_detection_methods = {
            'inverse_weighted': inverse_weighted_projection,
            'dark_spot': dark_spot_enhancement_projection,
            'junction_gap': junction_gap_projection,
            'hollow_structure': hollow_structure_projection,
            'diff_density': differential_density_projection,
            'phase_contrast': phase_contrast_simulation_projection,
            'local_minima': local_minima_projection,
            'texture_diff': texture_difference_projection,
            'vesicle_enhance': vesicle_enhancing_projection
        }

        # Apply each method and enhance result
        for method_name, method_func in vacuole_detection_methods.items():
            try:
                # Apply projection method
                print(f"  Applying {method_name} projection to channel {channel+1}...")
                proj = method_func(channel_data)

                # Apply CLAHE for better visualization
                enhanced = exposure.equalize_adapthist(proj, clip_limit=0.03)

                # Store result
                key = f"channel_{channel+1}_{method_name}"
                projections[key] = enhanced
                print(f"  Successfully created {key} projection")
            except Exception as e:
                print(f"  Error applying {method_name} projection: {e}")
                import traceback
                traceback.print_exc()

    return projections

# This function can then be integrated with your existing process_all_images() function

In [5]:
import numpy as np
from skimage import filters, morphology, feature, measure, exposure, img_as_float, img_as_uint
from scipy import ndimage
import matplotlib.pyplot as plt

def normalize_image(img):
    """Normalize image to 0-1 range"""
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:
        return (img - img_min) / (img_max - img_min)
    return img

def inverse_weighted_projection(image_stack, sigma=1.5):
    """
    Inverse-weighted projection that emphasizes dark regions (potential vacuoles)
    by giving higher weight to darker voxels in the z-direction
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Invert the stack to emphasize dark regions
    inverted_stack = 1 - stack

    # Calculate weights based on darkness (higher value = darker pixel)
    weights = filters.gaussian(inverted_stack, sigma=sigma)

    # Normalize weights for each pixel position
    weight_sum = np.sum(weights, axis=0)
    weight_sum[weight_sum == 0] = 1  # Avoid division by zero

    # Calculate weighted average of original (non-inverted) stack
    output = np.zeros_like(stack[0])
    for i in range(stack.shape[0]):
        output += stack[i] * weights[i] / weight_sum

    return output

def dark_spot_enhancement_projection(image_stack, spot_size=5):
    """
    Projects the stack while enhancing dark spots of a specific size range
    Useful for highlighting vacuoles/holes in cadherin-stained cells
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create output with maximum values first
    output = np.max(stack, axis=0)

    # Process each z-slice to detect dark spots
    for z in range(stack.shape[0]):
        # Detect dark spots using white top-hat transform on inverted image
        inverted = 1 - stack[z]
        selem = morphology.disk(spot_size)
        dark_spots = morphology.white_tophat(inverted, selem)

        # Threshold to identify significant dark spots
        threshold = filters.threshold_otsu(dark_spots) * 0.5
        dark_spots_binary = dark_spots > threshold

        # Label connected regions
        labeled_spots = measure.label(dark_spots_binary)

        # For each labeled region, check if it's a potential vacuole
        for region in measure.regionprops(labeled_spots):
            # Filter based on size and shape
            if region.area > 10 and region.area < 500 and region.eccentricity < 0.8:
                # Get region coordinates
                coords = region.coords

                # Update output image to show this dark spot
                for coord in coords:
                    y, x = coord
                    # Replace with dark value from this slice if it's darker
                    if stack[z, y, x] < output[y, x]:
                        output[y, x] = stack[z, y, x]

    return output

def junction_gap_projection(image_stack, membrane_thickness=3):
    """
    Specifically designed to detect gaps in cadherin staining at cell junctions
    by looking for discontinuities in membrane staining patterns
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Get maximum intensity projection as base
    mip = np.max(stack, axis=0)

    # Enhance edges to highlight cell boundaries
    edges = np.zeros_like(mip)
    for z in range(stack.shape[0]):
        # Detect edges using Sobel filter
        edge_sobel = filters.sobel(stack[z])
        edges = np.maximum(edges, edge_sobel)

    # Threshold edges to get membrane mask
    edge_threshold = filters.threshold_otsu(edges) * 0.5
    membrane_mask = edges > edge_threshold

    # Dilate to connect nearby membrane segments
    membrane_mask = morphology.binary_dilation(membrane_mask, morphology.disk(membrane_thickness))

    # Find potential gaps - areas where membrane should be but intensity is low
    gap_candidates = np.zeros_like(mip)

    # Process each z-slice to look for gaps in membranes
    for z in range(stack.shape[0]):
        # Get regions where membrane mask exists but pixel intensity is low
        low_intensity = stack[z] < np.percentile(stack[z][membrane_mask], 30)
        potential_gaps = membrane_mask & low_intensity

        # Add to candidates
        gap_candidates[potential_gaps] += 1

    # Normalize gap candidates
    gap_candidates = gap_candidates / stack.shape[0]

    # Create output that emphasizes gaps in junctions
    output = mip.copy()

    # Highlight gap regions by reducing intensity
    gap_regions = gap_candidates > 0.5
    output[gap_regions] = output[gap_regions] * 0.5

    return output

def hollow_structure_projection(image_stack, threshold=0.2, min_size=20, max_size=500):
    """
    Projects 3D hollow structures (potential vacuoles) by identifying dark regions
    surrounded by brighter regions across multiple z-slices
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Base projection - maximum intensity
    output = np.max(stack, axis=0)

    # Store potential hollow structures
    hollow_mask = np.zeros_like(output, dtype=bool)

    # Process each z slice
    for z in range(1, stack.shape[0]-1):  # Skip first and last slice
        # Threshold this slice to get foreground
        binary = stack[z] > threshold

        # Find holes in the binary image
        holes = morphology.remove_small_holes(binary, min_size) != binary

        # Remove small objects
        holes = morphology.remove_small_objects(holes, min_size)

        # Label connected components
        labeled_holes = measure.label(holes)

        # For each hole, check if it's surrounded by higher intensity in z-direction
        for region in measure.regionprops(labeled_holes):
            if region.area < max_size:  # Size constraint
                # Get region mask
                y0, x0, y1, x1 = region.bbox
                region_mask = labeled_holes[y0:y1, x0:x1] == region.label

                # Check intensity above and below
                above_intensity = np.mean(stack[min(z+1, stack.shape[0]-1), y0:y1, x0:x1][region_mask])
                below_intensity = np.mean(stack[max(z-1, 0), y0:y1, x0:x1][region_mask])
                current_intensity = np.mean(stack[z, y0:y1, x0:x1][region_mask])

                # If current slice is darker than slices above and below, it might be a hollow structure
                if current_intensity < above_intensity*0.8 and current_intensity < below_intensity*0.8:
                    hollow_mask[y0:y1, x0:x1] |= region_mask

    # Enhance the visualization by making hollow regions darker
    output[hollow_mask] = output[hollow_mask] * 0.3

    return output

def differential_density_projection(image_stack, kernel_size=5):
    """
    Highlights regions where there's significant difference in density/intensity
    between adjacent z-layers, useful for detecting vacuole boundaries
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Calculate gradient magnitude in z direction
    z_gradient = np.zeros_like(stack)
    for z in range(1, stack.shape[0]):
        z_gradient[z] = np.abs(stack[z] - stack[z-1])

    # Smooth the gradient
    smoothed_gradient = np.zeros_like(z_gradient)
    for z in range(z_gradient.shape[0]):
        smoothed_gradient[z] = filters.gaussian(z_gradient[z], sigma=1)

    # Get the maximum gradient projection
    max_gradient = np.max(smoothed_gradient, axis=0)

    # Create output that combines intensity and gradient information
    # Higher gradient indicates boundary of structure
    output = np.max(stack, axis=0)

    # Enhance regions with high gradient
    norm_gradient = normalize_image(max_gradient)
    output = output * (1 - norm_gradient * 0.5)

    return output

def phase_contrast_simulation_projection(image_stack, sigma=2):
    """
    Simulates phase contrast-like visualization to enhance membrane and vacuole boundaries
    by emphasizing intensity gradients in all directions
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create output image
    output = np.zeros_like(stack[0])

    # Process each z slice
    for z in range(stack.shape[0]):
        # Apply Laplacian of Gaussian filter to simulate phase contrast
        log_filtered = filters.laplace(filters.gaussian(stack[z], sigma=sigma))

        # Normalize to -1 to 1 range
        log_filtered = 2 * normalize_image(log_filtered) - 1

        # Convert to phase contrast-like appearance
        phase_contrast = 0.5 + 0.5 * log_filtered

        # Add to output based on original intensity (weighted)
        output += phase_contrast * stack[z]

    # Normalize
    output = normalize_image(output)

    return output

def local_minima_projection(image_stack, footprint_size=5):
    """
    Projects local minima across the z-stack, which could correspond to vacuoles
    or gaps in cadherin staining
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize with maximum values
    output = np.ones_like(stack[0])

    # Process each z slice
    for z in range(stack.shape[0]):
        # Find local minima
        footprint = morphology.disk(footprint_size)
        local_min = feature.peak_local_max(
            -stack[z],  # Invert to find minima
            footprint=footprint,
            indices=False
        )

        # For each local minimum, check if it's a potential vacuole
        labeled_mins = measure.label(local_min)

        for region in measure.regionprops(labeled_mins):
            y, x = region.centroid
            y, x = int(y), int(x)

            # Update output with local minimum value if it's lower
            if stack[z, y, x] < output[y, x]:
                # Update a small region around the minimum
                y_min, x_min = max(0, y-2), max(0, x-2)
                y_max, x_max = min(stack.shape[1], y+3), min(stack.shape[2], x+3)

                # Check each pixel in the neighborhood
                for yi in range(y_min, y_max):
                    for xi in range(x_min, x_max):
                        if stack[z, yi, xi] < output[yi, xi]:
                            output[yi, xi] = stack[z, yi, xi]

    return output

def vesicle_enhancing_projection(image_stack, min_size=5, max_size=50):
    """
    Specifically designed to enhance small vesicles or vacuoles
    by detecting spherical hollow structures across z-slices
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize outputs
    output = np.max(stack, axis=0)  # Base is maximum projection
    vesicle_mask = np.zeros_like(output, dtype=bool)

    # Step 1: Detect circular structures in each slice
    for z in range(stack.shape[0]):
        # Apply a white tophat filter to enhance small structures
        tophat = morphology.white_tophat(1-stack[z], morphology.disk(max_size//2))

        # Threshold to get potential vesicles
        thresh = filters.threshold_otsu(tophat) * 0.7
        binary = tophat > thresh

        # Remove small noise and fill holes
        binary = morphology.remove_small_objects(binary, min_size)
        binary = morphology.remove_small_holes(binary)

        # Label connected components
        labeled = measure.label(binary)

        # Analyze each region
        for region in measure.regionprops(labeled):
            # Filter based on size and circularity
            if (min_size < region.area < max_size and
                region.eccentricity < 0.6 and  # Circular shape
                region.solidity > 0.8):        # Solid/compact

                # Get coordinates
                y0, x0, y1, x1 = region.bbox
                mask = labeled[y0:y1, x0:x1] == region.label

                # Check intensity inside vs outside
                inside_intensity = np.mean(stack[z, y0:y1, x0:x1][mask])

                # Create dilated mask for outside region
                dilated_mask = morphology.binary_dilation(mask, morphology.disk(2))
                dilated_mask = dilated_mask & ~mask

                # Check if there's enough border pixels
                if np.sum(dilated_mask) > 0:
                    outside_intensity = np.mean(stack[z, y0:y1, x0:x1][dilated_mask])

                    # If inside is significantly darker than outside, it's a potential vesicle
                    if inside_intensity < outside_intensity * 0.7:
                        vesicle_mask[y0:y1, x0:x1] |= mask

    # Enhance visualization by darkening detected vesicles
    output[vesicle_mask] = output[vesicle_mask] * 0.2

    # And brightening their boundaries for better visibility
    vesicle_boundary = morphology.binary_dilation(vesicle_mask) & ~vesicle_mask
    output[vesicle_boundary] = np.minimum(output[vesicle_boundary] * 1.5, 1.0)

    return output

# Example usage:

# Replace this with your actual image loading code
import tifffile

# Load a 3D stack
image_stack = tifffile.imread('your_image.tif')

# If the image has multiple channels, select the channel for cadherin staining
if len(image_stack.shape) == 4:  # (channels, z, y, x)
    cadherin_channel = 0  # Replace with your cadherin channel index
    channel_data = image_stack[cadherin_channel]
elif len(image_stack.shape) == 3:  # (z, y, x)
    channel_data = image_stack

# Apply projections
inverse_weighted_result = inverse_weighted_projection(channel_data)
dark_spot_result = dark_spot_enhancement_projection(channel_data)
junction_gap_result = junction_gap_projection(channel_data)
hollow_structure_result = hollow_structure_projection(channel_data)
diff_density_result = differential_density_projection(channel_data)
phase_contrast_result = phase_contrast_simulation_projection(channel_data)
local_minima_result = local_minima_projection(channel_data)
vesicle_result = vesicle_enhancing_projection(channel_data)

# Save results
tifffile.imwrite('inverse_weighted.tif', img_as_uint(inverse_weighted_result))
tifffile.imwrite('dark_spot.tif', img_as_uint(dark_spot_result))
tifffile.imwrite('junction_gap.tif', img_as_uint(junction_gap_result))
tifffile.imwrite('hollow_structure.tif', img_as_uint(hollow_structure_result))
tifffile.imwrite('diff_density.tif', img_as_uint(diff_density_result))
tifffile.imwrite('phase_contrast.tif', img_as_uint(phase_contrast_result))
tifffile.imwrite('local_minima.tif', img_as_uint(local_minima_result))
tifffile.imwrite('vesicle.tif', img_as_uint(vesicle_result))

# Visualize
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))
projections = {
    'Original MIP': np.max(channel_data, axis=0),
    'Inverse Weighted': inverse_weighted_result,
    'Dark Spot': dark_spot_result,
    'Junction Gap': junction_gap_result,
    'Hollow Structure': hollow_structure_result,
    'Differential Density': diff_density_result,
    'Phase Contrast': phase_contrast_result,
    'Local Minima': local_minima_result,
    'Vesicle Enhancing': vesicle_result
}

for i, (name, projection) in enumerate(projections.items()):
    plt.subplot(3, 3, i+1)
    plt.imshow(projection, cmap='gray')
    plt.title(name)
    plt.axis('off')

plt.tight_layout()
plt.show()


"\n# Replace this with your actual image loading code\nimport tifffile\n\n# Load a 3D stack\nimage_stack = tifffile.imread('your_image.tif')\n\n# If the image has multiple channels, select the channel for cadherin staining\nif len(image_stack.shape) == 4:  # (channels, z, y, x)\n    cadherin_channel = 0  # Replace with your cadherin channel index\n    channel_data = image_stack[cadherin_channel]\nelif len(image_stack.shape) == 3:  # (z, y, x)\n    channel_data = image_stack\n\n# Apply projections\ninverse_weighted_result = inverse_weighted_projection(channel_data)\ndark_spot_result = dark_spot_enhancement_projection(channel_data)\njunction_gap_result = junction_gap_projection(channel_data)\nhollow_structure_result = hollow_structure_projection(channel_data)\ndiff_density_result = differential_density_projection(channel_data)\nphase_contrast_result = phase_contrast_simulation_projection(channel_data)\nlocal_minima_result = local_minima_projection(channel_data)\nvesicle_result = vesicle_

In [6]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
from skimage import filters, morphology, feature, measure, exposure, img_as_float, img_as_uint
from scipy import ndimage
import matplotlib.pyplot as plt
import glob
import tifffile

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/trial22'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def normalize_image(img):
    """Normalize image to 0-1 range"""
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:
        return (img - img_min) / (img_max - img_min)
    return img

def inverse_weighted_projection(image_stack, sigma=1.5):
    """
    Inverse-weighted projection that emphasizes dark regions (potential vacuoles)
    by giving higher weight to darker voxels in the z-direction
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Invert the stack to emphasize dark regions
    inverted_stack = 1 - stack

    # Calculate weights based on darkness (higher value = darker pixel)
    weights = filters.gaussian(inverted_stack, sigma=sigma)

    # Normalize weights for each pixel position
    weight_sum = np.sum(weights, axis=0)
    weight_sum[weight_sum == 0] = 1  # Avoid division by zero

    # Calculate weighted average of original (non-inverted) stack
    output = np.zeros_like(stack[0])
    for i in range(stack.shape[0]):
        output += stack[i] * weights[i] / weight_sum

    return output

def dark_spot_enhancement_projection(image_stack, spot_size=5):
    """
    Projects the stack while enhancing dark spots of a specific size range
    Useful for highlighting vacuoles/holes in cadherin-stained cells
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create output with maximum values first
    output = np.max(stack, axis=0)

    # Process each z-slice to detect dark spots
    for z in range(stack.shape[0]):
        # Detect dark spots using white top-hat transform on inverted image
        inverted = 1 - stack[z]
        selem = morphology.disk(spot_size)
        dark_spots = morphology.white_tophat(inverted, selem)

        # Threshold to identify significant dark spots
        threshold = filters.threshold_otsu(dark_spots) * 0.5
        dark_spots_binary = dark_spots > threshold

        # Label connected regions
        labeled_spots = measure.label(dark_spots_binary)

        # For each labeled region, check if it's a potential vacuole
        for region in measure.regionprops(labeled_spots):
            # Filter based on size and shape
            if region.area > 10 and region.area < 500 and region.eccentricity < 0.8:
                # Get region coordinates
                coords = region.coords

                # Update output image to show this dark spot
                for coord in coords:
                    y, x = coord
                    # Replace with dark value from this slice if it's darker
                    if stack[z, y, x] < output[y, x]:
                        output[y, x] = stack[z, y, x]

    return output

def junction_gap_projection(image_stack, membrane_thickness=3):
    """
    Specifically designed to detect gaps in cadherin staining at cell junctions
    by looking for discontinuities in membrane staining patterns
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Get maximum intensity projection as base
    mip = np.max(stack, axis=0)

    # Enhance edges to highlight cell boundaries
    edges = np.zeros_like(mip)
    for z in range(stack.shape[0]):
        # Detect edges using Sobel filter
        edge_sobel = filters.sobel(stack[z])
        edges = np.maximum(edges, edge_sobel)

    # Threshold edges to get membrane mask
    edge_threshold = filters.threshold_otsu(edges) * 0.5
    membrane_mask = edges > edge_threshold

    # Dilate to connect nearby membrane segments
    membrane_mask = morphology.binary_dilation(membrane_mask, morphology.disk(membrane_thickness))

    # Find potential gaps - areas where membrane should be but intensity is low
    gap_candidates = np.zeros_like(mip)

    # Process each z-slice to look for gaps in membranes
    for z in range(stack.shape[0]):
        # Get regions where membrane mask exists but pixel intensity is low
        low_intensity = stack[z] < np.percentile(stack[z][membrane_mask], 30)
        potential_gaps = membrane_mask & low_intensity

        # Add to candidates
        gap_candidates[potential_gaps] += 1

    # Normalize gap candidates
    gap_candidates = gap_candidates / stack.shape[0]

    # Create output that emphasizes gaps in junctions
    output = mip.copy()

    # Highlight gap regions by reducing intensity
    gap_regions = gap_candidates > 0.5
    output[gap_regions] = output[gap_regions] * 0.5

    return output

def hollow_structure_projection(image_stack, threshold=0.2, min_size=20, max_size=500):
    """
    Projects 3D hollow structures (potential vacuoles) by identifying dark regions
    surrounded by brighter regions across multiple z-slices
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Base projection - maximum intensity
    output = np.max(stack, axis=0)

    # Store potential hollow structures
    hollow_mask = np.zeros_like(output, dtype=bool)

    # Process each z slice
    for z in range(1, stack.shape[0]-1):  # Skip first and last slice
        # Threshold this slice to get foreground
        binary = stack[z] > threshold

        # Find holes in the binary image
        holes = morphology.remove_small_holes(binary, min_size) != binary

        # Remove small objects
        holes = morphology.remove_small_objects(holes, min_size)

        # Label connected components
        labeled_holes = measure.label(holes)

        # For each hole, check if it's surrounded by higher intensity in z-direction
        for region in measure.regionprops(labeled_holes):
            if region.area < max_size:  # Size constraint
                # Get region mask
                y0, x0, y1, x1 = region.bbox
                region_mask = labeled_holes[y0:y1, x0:x1] == region.label

                # Check intensity above and below
                above_intensity = np.mean(stack[min(z+1, stack.shape[0]-1), y0:y1, x0:x1][region_mask])
                below_intensity = np.mean(stack[max(z-1, 0), y0:y1, x0:x1][region_mask])
                current_intensity = np.mean(stack[z, y0:y1, x0:x1][region_mask])

                # If current slice is darker than slices above and below, it might be a hollow structure
                if current_intensity < above_intensity*0.8 and current_intensity < below_intensity*0.8:
                    hollow_mask[y0:y1, x0:x1] |= region_mask

    # Enhance the visualization by making hollow regions darker
    output[hollow_mask] = output[hollow_mask] * 0.3

    return output

def differential_density_projection(image_stack, kernel_size=5):
    """
    Highlights regions where there's significant difference in density/intensity
    between adjacent z-layers, useful for detecting vacuole boundaries
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Calculate gradient magnitude in z direction
    z_gradient = np.zeros_like(stack)
    for z in range(1, stack.shape[0]):
        z_gradient[z] = np.abs(stack[z] - stack[z-1])

    # Smooth the gradient
    smoothed_gradient = np.zeros_like(z_gradient)
    for z in range(z_gradient.shape[0]):
        smoothed_gradient[z] = filters.gaussian(z_gradient[z], sigma=1)

    # Get the maximum gradient projection
    max_gradient = np.max(smoothed_gradient, axis=0)

    # Create output that combines intensity and gradient information
    # Higher gradient indicates boundary of structure
    output = np.max(stack, axis=0)

    # Enhance regions with high gradient
    norm_gradient = normalize_image(max_gradient)
    output = output * (1 - norm_gradient * 0.5)

    return output

def phase_contrast_simulation_projection(image_stack, sigma=2):
    """
    Simulates phase contrast-like visualization to enhance membrane and vacuole boundaries
    by emphasizing intensity gradients in all directions
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create output image
    output = np.zeros_like(stack[0])

    # Process each z slice
    for z in range(stack.shape[0]):
        # Apply Laplacian of Gaussian filter to simulate phase contrast
        log_filtered = filters.laplace(filters.gaussian(stack[z], sigma=sigma))

        # Normalize to -1 to 1 range
        log_filtered = 2 * normalize_image(log_filtered) - 1

        # Convert to phase contrast-like appearance
        phase_contrast = 0.5 + 0.5 * log_filtered

        # Add to output based on original intensity (weighted)
        output += phase_contrast * stack[z]

    # Normalize
    output = normalize_image(output)

    return output

def local_minima_projection(image_stack, footprint_size=5):
    """
    Projects local minima across the z-stack, which could correspond to vacuoles
    or gaps in cadherin staining
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize with maximum values
    output = np.ones_like(stack[0])

    # Process each z slice
    for z in range(stack.shape[0]):
        # Find local minima
        footprint = morphology.disk(footprint_size)
        local_min = feature.peak_local_max(
            -stack[z],  # Invert to find minima
            footprint=footprint,
            indices=False
        )

        # For each local minimum, check if it's a potential vacuole
        labeled_mins = measure.label(local_min)

        for region in measure.regionprops(labeled_mins):
            y, x = region.centroid
            y, x = int(y), int(x)

            # Update output with local minimum value if it's lower
            if stack[z, y, x] < output[y, x]:
                # Update a small region around the minimum
                y_min, x_min = max(0, y-2), max(0, x-2)
                y_max, x_max = min(stack.shape[1], y+3), min(stack.shape[2], x+3)

                # Check each pixel in the neighborhood
                for yi in range(y_min, y_max):
                    for xi in range(x_min, x_max):
                        if stack[z, yi, xi] < output[yi, xi]:
                            output[yi, xi] = stack[z, yi, xi]

    return output

def vesicle_enhancing_projection(image_stack, min_size=5, max_size=50):
    """
    Specifically designed to enhance small vesicles or vacuoles
    by detecting spherical hollow structures across z-slices
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize outputs
    output = np.max(stack, axis=0)  # Base is maximum projection
    vesicle_mask = np.zeros_like(output, dtype=bool)

    # Step 1: Detect circular structures in each slice
    for z in range(stack.shape[0]):
        # Apply a white tophat filter to enhance small structures
        tophat = morphology.white_tophat(1-stack[z], morphology.disk(max_size//2))

        # Threshold to get potential vesicles
        thresh = filters.threshold_otsu(tophat) * 0.7
        binary = tophat > thresh

        # Remove small noise and fill holes
        binary = morphology.remove_small_objects(binary, min_size)
        binary = morphology.remove_small_holes(binary)

        # Label connected components
        labeled = measure.label(binary)

        # Analyze each region
        for region in measure.regionprops(labeled):
            # Filter based on size and circularity
            if (min_size < region.area < max_size and
                region.eccentricity < 0.6 and  # Circular shape
                region.solidity > 0.8):        # Solid/compact

                # Get coordinates
                y0, x0, y1, x1 = region.bbox
                mask = labeled[y0:y1, x0:x1] == region.label

                # Check intensity inside vs outside
                inside_intensity = np.mean(stack[z, y0:y1, x0:x1][mask])

                # Create dilated mask for outside region
                dilated_mask = morphology.binary_dilation(mask, morphology.disk(2))
                dilated_mask = dilated_mask & ~mask

                # Check if there's enough border pixels
                if np.sum(dilated_mask) > 0:
                    outside_intensity = np.mean(stack[z, y0:y1, x0:x1][dilated_mask])

                    # If inside is significantly darker than outside, it's a potential vesicle
                    if inside_intensity < outside_intensity * 0.7:
                        vesicle_mask[y0:y1, x0:x1] |= mask

    # Enhance visualization by darkening detected vesicles
    output[vesicle_mask] = output[vesicle_mask] * 0.2

    # And brightening their boundaries for better visibility
    vesicle_boundary = morphology.binary_dilation(vesicle_mask) & ~vesicle_mask
    output[vesicle_boundary] = np.minimum(output[vesicle_boundary] * 1.5, 1.0)

    return output

def process_4d_microscopy_image(image, filename, process_first_channel_only=True):
    """
    Process a 4D microscopy image with new projection methods for vacuole detection

    Parameters:
    -----------
    image : numpy.ndarray
        4D input image with shape (channels, z-slices, height, width)
    filename : str
        Original filename for naming the output
    process_first_channel_only : bool
        If True, only process the first channel

    Returns:
    --------
    dict
        Dictionary of processed images
    """
    print(f"  Processing 4D image with shape {image.shape}")

    # Normalize values if needed
    if image.dtype == np.float32 or image.dtype == np.float64:
        if np.max(image) > 1.0 or np.min(image) < -1.0:
            print(f"  Normalizing image values from range [{np.min(image)}, {np.max(image)}] to [0, 1]")
            for c in range(image.shape[0]):
                for z in range(image.shape[1]):
                    image[c, z] = normalize_image(image[c, z])

    projections = {}

    # Determine which channels to process
    channels_to_process = [0] if process_first_channel_only else range(image.shape[0])

    # Process each channel
    for channel in channels_to_process:
        # Get all z-slices for this channel
        channel_data = image[channel]

        # Define vacuole/gap detection projection methods
        vacuole_detection_methods = {
            'inverse_weighted': inverse_weighted_projection,
            'dark_spot': dark_spot_enhancement_projection,
            'junction_gap': junction_gap_projection,
            'hollow_structure': hollow_structure_projection,
            'diff_density': differential_density_projection,
            'phase_contrast': phase_contrast_simulation_projection,
            'local_minima': local_minima_projection,
            'vesicle_enhance': vesicle_enhancing_projection
        }

        # Apply each method and enhance result
        for method_name, method_func in vacuole_detection_methods.items():
            try:
                # Apply projection method
                print(f"  Applying {method_name} projection to channel {channel+1}...")
                proj = method_func(channel_data)

                # Apply CLAHE for better visualization
                enhanced = exposure.equalize_adapthist(proj, clip_limit=0.03)

                # Store result
                key = f"channel_{channel+1}_{method_name}"
                projections[key] = enhanced
                print(f"  Successfully created {key} projection")
            except Exception as e:
                print(f"  Error applying {method_name} projection: {e}")
                import traceback
                traceback.print_exc()

    return projections

def process_all_images():
    """
    Process all 4D microscopy images in the input directory
    """
    # Get all image files with common extensions (case insensitive)
    extensions = ['.tif', '.tiff', '.TIF', '.TIFF']
    all_files = []

    for ext in extensions:
        all_files.extend(glob.glob(os.path.join(input_dir, f'*{ext}')))

    if len(all_files) == 0:
        print(f"No image files found in {input_dir}")
        return []

    # Sort files to ensure consistent processing order
    all_files.sort()

    print(f"Found {len(all_files)} image files. Processing each with multiple methods...")

    processed_files = []

    # Process each image
    for idx, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        print(f"Processing image {idx+1}/{len(all_files)}: {filename}")

        try:
            # Load the image
            image = tifffile.imread(file_path)

            # Print image info
            print(f"  Image shape: {image.shape}, dtype: {image.dtype}")

            # Skip if not 4D
            if len(image.shape) != 4:
                print(f"  Skipping - not a 4D image")
                continue

            # Process the 4D image (only first channel)
            projections = process_4d_microscopy_image(image, filename, process_first_channel_only=True)

            # Save each projection
            for proj_name, projection in projections.items():
                # Create output filename
                base_name = os.path.splitext(filename)[0]
                output_filename = f"{base_name}_{proj_name}.tif"
                output_path = os.path.join(output_dir, output_filename)

                # Convert to uint16 for saving
                if projection.dtype == np.float64 or projection.dtype == np.float32:
                    projection = img_as_uint(projection)

                # Save the projection
                tifffile.imwrite(output_path, projection)

                # Add to list of processed files
                processed_files.append(output_path)

            print(f"  Saved {len(projections)} projections")

        except Exception as e:
            print(f"  Error processing image {filename}: {e}")
            import traceback
            traceback.print_exc()
            continue

    return processed_files

# Main execution
print("Starting 4D image processing with vacuole detection projection methods...")
print(f"Input directory: {input_dir}")
print(f"Output directory: {output_dir}")

# Process all image files in the directory
processed_files = process_all_images()

# Display results
if processed_files:
    print(f"\nSuccessfully processed {len(processed_files)} projections:")
    for output_file in processed_files[:5]:  # Show first 5
        print(f"  - {os.path.basename(output_file)}")
    if len(processed_files) > 5:
        print(f"  ... and {len(processed_files) - 5} more")

    # Try to display the first result in the notebook
    try:
        from IPython.display import Image
        print("\nDisplaying first result:")
        Image(processed_files[0])
    except Exception as e:
        print(f"Couldn't display image: {e}")
else:
    print("No images were processed successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting 4D image processing with vacuole detection projection methods...
Input directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial
Output directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/trial22
Found 1 image files. Processing each with multiple methods...
Processing image 1/1: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1).tif
  Image shape: (3, 21, 1024, 1024), dtype: float32
  Processing 4D image with shape (3, 21, 1024, 1024)
  Normalizing image values from range [-0.02250508777797222, 3.4323275089263916] to [0, 1]
  Applying inverse_weighted projection to channel 1...
  Successfully created channel_1_inverse_weighted projection
  Applying dark_spot projection to channel 1...
  Successfully created channel_1_dark_spot projection
  Applying junction_gap projection to channel 1...
  Success

Traceback (most recent call last):
  File "<ipython-input-6-f0e468769a25>", line 413, in process_4d_microscopy_image
    proj = method_func(channel_data)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-6-f0e468769a25>", line 268, in local_minima_projection
    local_min = feature.peak_local_max(
                ^^^^^^^^^^^^^^^^^^^^^^^
TypeError: peak_local_max() got an unexpected keyword argument 'indices'


  Successfully created channel_1_vesicle_enhance projection
  Saved 7 projections

Successfully processed 7 projections:
  - denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1)_channel_1_inverse_weighted.tif
  - denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1)_channel_1_dark_spot.tif
  - denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1)_channel_1_junction_gap.tif
  - denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1)_channel_1_hollow_structure.tif
  - denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1)_channel_1_diff_density.tif
  ... and 2 more

Displaying first result:
Couldn't display image: Cannot embed the 'tif' image format


In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
from skimage import filters, morphology, feature, measure, exposure, img_as_float, img_as_uint, segmentation
from scipy import ndimage
import matplotlib.pyplot as plt
import glob
import tifffile
import traceback

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/trial23'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def normalize_image(img):
    """Normalize image to 0-1 range"""
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:
        return (img - img_min) / (img_max - img_min)
    return img

# Original projection methods
def inverse_weighted_projection(image_stack, sigma=1.5):
    """
    Inverse-weighted projection that emphasizes dark regions (potential vacuoles)
    by giving higher weight to darker voxels in the z-direction
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Invert the stack to emphasize dark regions
    inverted_stack = 1 - stack

    # Calculate weights based on darkness (higher value = darker pixel)
    weights = filters.gaussian(inverted_stack, sigma=sigma)

    # Normalize weights for each pixel position
    weight_sum = np.sum(weights, axis=0)
    weight_sum[weight_sum == 0] = 1  # Avoid division by zero

    # Calculate weighted average of original (non-inverted) stack
    output = np.zeros_like(stack[0])
    for i in range(stack.shape[0]):
        output += stack[i] * weights[i] / weight_sum

    return output

def dark_spot_enhancement_projection(image_stack, spot_size=5):
    """
    Projects the stack while enhancing dark spots of a specific size range
    Useful for highlighting vacuoles/holes in cadherin-stained cells
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create output with maximum values first
    output = np.max(stack, axis=0)

    # Process each z-slice to detect dark spots
    for z in range(stack.shape[0]):
        # Detect dark spots using white top-hat transform on inverted image
        inverted = 1 - stack[z]
        selem = morphology.disk(spot_size)
        dark_spots = morphology.white_tophat(inverted, selem)

        # Threshold to identify significant dark spots
        threshold = filters.threshold_otsu(dark_spots) * 0.5
        dark_spots_binary = dark_spots > threshold

        # Label connected regions
        labeled_spots = measure.label(dark_spots_binary)

        # For each labeled region, check if it's a potential vacuole
        for region in measure.regionprops(labeled_spots):
            # Filter based on size and shape
            if region.area > 10 and region.area < 500 and region.eccentricity < 0.8:
                # Get region coordinates
                coords = region.coords

                # Update output image to show this dark spot
                for coord in coords:
                    y, x = coord
                    # Replace with dark value from this slice if it's darker
                    if stack[z, y, x] < output[y, x]:
                        output[y, x] = stack[z, y, x]

    return output

def junction_gap_projection(image_stack, membrane_thickness=3):
    """
    Specifically designed to detect gaps in cadherin staining at cell junctions
    by looking for discontinuities in membrane staining patterns
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Get maximum intensity projection as base
    mip = np.max(stack, axis=0)

    # Enhance edges to highlight cell boundaries
    edges = np.zeros_like(mip)
    for z in range(stack.shape[0]):
        # Detect edges using Sobel filter
        edge_sobel = filters.sobel(stack[z])
        edges = np.maximum(edges, edge_sobel)

    # Threshold edges to get membrane mask
    edge_threshold = filters.threshold_otsu(edges) * 0.5
    membrane_mask = edges > edge_threshold

    # Dilate to connect nearby membrane segments
    membrane_mask = morphology.binary_dilation(membrane_mask, morphology.disk(membrane_thickness))

    # Find potential gaps - areas where membrane should be but intensity is low
    gap_candidates = np.zeros_like(mip)

    # Process each z-slice to look for gaps in membranes
    for z in range(stack.shape[0]):
        # Get regions where membrane mask exists but pixel intensity is low
        low_intensity = stack[z] < np.percentile(stack[z][membrane_mask], 30)
        potential_gaps = membrane_mask & low_intensity

        # Add to candidates
        gap_candidates[potential_gaps] += 1

    # Normalize gap candidates
    gap_candidates = gap_candidates / stack.shape[0]

    # Create output that emphasizes gaps in junctions
    output = mip.copy()

    # Highlight gap regions by reducing intensity
    gap_regions = gap_candidates > 0.5
    output[gap_regions] = output[gap_regions] * 0.5

    return output

def hollow_structure_projection(image_stack, threshold=0.2, min_size=20, max_size=500):
    """
    Projects 3D hollow structures (potential vacuoles) by identifying dark regions
    surrounded by brighter regions across multiple z-slices
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Base projection - maximum intensity
    output = np.max(stack, axis=0)

    # Store potential hollow structures
    hollow_mask = np.zeros_like(output, dtype=bool)

    # Process each z slice
    for z in range(1, stack.shape[0]-1):  # Skip first and last slice
        # Threshold this slice to get foreground
        binary = stack[z] > threshold

        # Find holes in the binary image
        holes = morphology.remove_small_holes(binary, min_size) != binary

        # Remove small objects
        holes = morphology.remove_small_objects(holes, min_size)

        # Label connected components
        labeled_holes = measure.label(holes)

        # For each hole, check if it's surrounded by higher intensity in z-direction
        for region in measure.regionprops(labeled_holes):
            if region.area < max_size:  # Size constraint
                # Get region mask
                y0, x0, y1, x1 = region.bbox
                region_mask = labeled_holes[y0:y1, x0:x1] == region.label

                # Check intensity above and below
                above_intensity = np.mean(stack[min(z+1, stack.shape[0]-1), y0:y1, x0:x1][region_mask])
                below_intensity = np.mean(stack[max(z-1, 0), y0:y1, x0:x1][region_mask])
                current_intensity = np.mean(stack[z, y0:y1, x0:x1][region_mask])

                # If current slice is darker than slices above and below, it might be a hollow structure
                if current_intensity < above_intensity*0.8 and current_intensity < below_intensity*0.8:
                    hollow_mask[y0:y1, x0:x1] |= region_mask

    # Enhance the visualization by making hollow regions darker
    output[hollow_mask] = output[hollow_mask] * 0.3

    return output

def differential_density_projection(image_stack, kernel_size=5):
    """
    Highlights regions where there's significant difference in density/intensity
    between adjacent z-layers, useful for detecting vacuole boundaries
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Calculate gradient magnitude in z direction
    z_gradient = np.zeros_like(stack)
    for z in range(1, stack.shape[0]):
        z_gradient[z] = np.abs(stack[z] - stack[z-1])

    # Smooth the gradient
    smoothed_gradient = np.zeros_like(z_gradient)
    for z in range(z_gradient.shape[0]):
        smoothed_gradient[z] = filters.gaussian(z_gradient[z], sigma=1)

    # Get the maximum gradient projection
    max_gradient = np.max(smoothed_gradient, axis=0)

    # Create output that combines intensity and gradient information
    # Higher gradient indicates boundary of structure
    output = np.max(stack, axis=0)

    # Enhance regions with high gradient
    norm_gradient = normalize_image(max_gradient)
    output = output * (1 - norm_gradient * 0.5)

    return output

def phase_contrast_simulation_projection(image_stack, sigma=2):
    """
    Simulates phase contrast-like visualization to enhance membrane and vacuole boundaries
    by emphasizing intensity gradients in all directions
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create output image
    output = np.zeros_like(stack[0])

    # Process each z slice
    for z in range(stack.shape[0]):
        # Apply Laplacian of Gaussian filter to simulate phase contrast
        log_filtered = filters.laplace(filters.gaussian(stack[z], sigma=sigma))

        # Normalize to -1 to 1 range
        log_filtered = 2 * normalize_image(log_filtered) - 1

        # Convert to phase contrast-like appearance
        phase_contrast = 0.5 + 0.5 * log_filtered

        # Add to output based on original intensity (weighted)
        output += phase_contrast * stack[z]

    # Normalize
    output = normalize_image(output)

    return output

def local_minima_projection(image_stack, footprint_size=5):
    """
    Projects local minima across the z-stack, which could correspond to vacuoles
    or gaps in cadherin staining
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize with maximum values
    output = np.ones_like(stack[0])

    # Process each z slice
    for z in range(stack.shape[0]):
        # Find local minima
        footprint = morphology.disk(footprint_size)
        local_min = feature.peak_local_max(
            -stack[z],  # Invert to find minima
            footprint=footprint,
            indices=False
        )

        # For each local minimum, check if it's a potential vacuole
        labeled_mins = measure.label(local_min)

        for region in measure.regionprops(labeled_mins):
            y, x = region.centroid
            y, x = int(y), int(x)

            # Update output with local minimum value if it's lower
            if stack[z, y, x] < output[y, x]:
                # Update a small region around the minimum
                y_min, x_min = max(0, y-2), max(0, x-2)
                y_max, x_max = min(stack.shape[1], y+3), min(stack.shape[2], x+3)

                # Check each pixel in the neighborhood
                for yi in range(y_min, y_max):
                    for xi in range(x_min, x_max):
                        if stack[z, yi, xi] < output[yi, xi]:
                            output[yi, xi] = stack[z, yi, xi]

    return output

def vesicle_enhancing_projection(image_stack, min_size=5, max_size=50):
    """
    Specifically designed to enhance small vesicles or vacuoles
    by detecting spherical hollow structures across z-slices
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize outputs
    output = np.max(stack, axis=0)  # Base is maximum projection
    vesicle_mask = np.zeros_like(output, dtype=bool)

    # Step 1: Detect circular structures in each slice
    for z in range(stack.shape[0]):
        # Apply a white tophat filter to enhance small structures
        tophat = morphology.white_tophat(1-stack[z], morphology.disk(max_size//2))

        # Threshold to get potential vesicles
        thresh = filters.threshold_otsu(tophat) * 0.7
        binary = tophat > thresh

        # Remove small noise and fill holes
        binary = morphology.remove_small_objects(binary, min_size)
        binary = morphology.remove_small_holes(binary)

        # Label connected components
        labeled = measure.label(binary)

        # Analyze each region
        for region in measure.regionprops(labeled):
            # Filter based on size and circularity
            if (min_size < region.area < max_size and
                region.eccentricity < 0.6 and  # Circular shape
                region.solidity > 0.8):        # Solid/compact

                # Get coordinates
                y0, x0, y1, x1 = region.bbox
                mask = labeled[y0:y1, x0:x1] == region.label

                # Check intensity inside vs outside
                inside_intensity = np.mean(stack[z, y0:y1, x0:x1][mask])

                # Create dilated mask for outside region
                dilated_mask = morphology.binary_dilation(mask, morphology.disk(2))
                dilated_mask = dilated_mask & ~mask

                # Check if there's enough border pixels
                if np.sum(dilated_mask) > 0:
                    outside_intensity = np.mean(stack[z, y0:y1, x0:x1][dilated_mask])

                    # If inside is significantly darker than outside, it's a potential vesicle
                    if inside_intensity < outside_intensity * 0.7:
                        vesicle_mask[y0:y1, x0:x1] |= mask

    # Enhance visualization by darkening detected vesicles
    output[vesicle_mask] = output[vesicle_mask] * 0.2

    # And brightening their boundaries for better visibility
    vesicle_boundary = morphology.binary_dilation(vesicle_mask) & ~vesicle_mask
    output[vesicle_boundary] = np.minimum(output[vesicle_boundary] * 1.5, 1.0)

    return output

# New Advanced Projection Methods

def texture_contrast_projection(image_stack, window_size=15, percentile=10):
    """
    Enhances holes by analyzing local texture contrast across z-slices.
    This method is particularly effective at distinguishing holes from other dark regions
    by measuring texture homogeneity.
    """
    from skimage.feature import local_binary_pattern

    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize outputs
    texture_variance = np.zeros_like(stack[0])
    intensity_min = np.ones_like(stack[0])

    # For each z-slice, calculate texture features
    for z in range(stack.shape[0]):
        # Calculate local binary pattern for texture
        radius = 3  # Radius for LBP calculation
        n_points = 8 * radius
        lbp = local_binary_pattern(stack[z], n_points, radius, 'uniform')

        # Calculate local variance of LBP (measure of texture complexity)
        lbp_mean = ndimage.uniform_filter(lbp, size=window_size)
        lbp_sqr_mean = ndimage.uniform_filter(lbp**2, size=window_size)
        lbp_variance = lbp_sqr_mean - lbp_mean**2

        # Normalize variance
        if np.max(lbp_variance) > 0:
            lbp_variance = lbp_variance / np.max(lbp_variance)

        # Update texture variance - we want maximum variance across all z-slices
        texture_variance = np.maximum(texture_variance, lbp_variance)

        # Track minimum intensity (likely holes)
        intensity_min = np.minimum(intensity_min, stack[z])

    # Identify regions with low texture variance (potentially homogeneous holes)
    low_texture_mask = texture_variance < np.percentile(texture_variance, percentile)

    # Combine with intensity information - holes are dark AND have low texture variance
    dark_regions = intensity_min < np.percentile(intensity_min, 20)

    # Potential holes are areas with both low texture and low intensity
    hole_candidates = low_texture_mask & dark_regions

    # Create output by enhancing hole regions
    output = np.max(stack, axis=0)  # Start with maximum projection

    # Enhance visualization by darkening detected holes
    output[hole_candidates] = output[hole_candidates] * 0.3

    # Enhance contrast
    output = exposure.rescale_intensity(output)

    return output

def directional_gradient_flow_projection(image_stack, sigma=2.0):
    """
    Projects areas where the directional gradient flow indicates enclosed spaces,
    which is characteristic of holes surrounded by bright cell membranes.

    This method analyzes gradient vector fields to find areas where vectors point inward,
    indicating potential enclosed holes.
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize accumulator for gradient flow convergence
    flow_convergence = np.zeros_like(stack[0])

    # Process each z-slice
    for z in range(stack.shape[0]):
        # Smooth the image
        smoothed = filters.gaussian(stack[z], sigma=sigma)

        # Calculate gradients
        gy, gx = np.gradient(smoothed)

        # Calculate gradient magnitude
        magnitude = np.sqrt(gx**2 + gy**2)

        # Normalize gradient vectors
        with np.errstate(divide='ignore', invalid='ignore'):
            gx_norm = np.where(magnitude > 0, gx / magnitude, 0)
            gy_norm = np.where(magnitude > 0, gy / magnitude, 0)

        # Calculate divergence (negative for convergence points)
        gxx, _ = np.gradient(gx_norm)
        _, gyy = np.gradient(gy_norm)
        divergence = gxx + gyy

        # Negative divergence means convergence (potential holes)
        convergence = -divergence
        convergence[convergence < 0] = 0  # Keep only convergence points

        # Add to accumulator
        flow_convergence = np.maximum(flow_convergence, convergence)

    # Normalize convergence
    if np.max(flow_convergence) > 0:
        flow_convergence = flow_convergence / np.max(flow_convergence)

    # Create output by combining maximum intensity projection with convergence information
    output = np.max(stack, axis=0)

    # Enhance areas with high convergence (potential holes)
    # Darkening regions with high convergence
    output = output * (1.0 - 0.7 * flow_convergence)

    # Enhance contrast
    output = exposure.rescale_intensity(output)

    return output

def multiscale_hole_detection_projection(image_stack, scales=[2, 4, 8, 16]):
    """
    Detects holes at multiple scales using Laplacian of Gaussian filters
    and combines the information across z-slices.

    This method is effective at finding holes of different sizes by analyzing
    blob-like structures at multiple scales simultaneously.
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize blob response
    blob_response = np.zeros_like(stack[0])

    # Process each z-slice
    for z in range(stack.shape[0]):
        # Invert image (so holes are bright)
        inverted = 1 - stack[z]

        # Apply LoG filter at multiple scales
        for sigma in scales:
            # Calculate LoG response
            log_response = feature.blob_log(
                inverted,
                min_sigma=sigma,
                max_sigma=sigma,
                num_sigma=1,
                threshold=0.02
            )

            # Create response image
            if len(log_response) > 0:
                temp_response = np.zeros_like(inverted)
                for blob in log_response:
                    y, x, s = blob
                    y, x = int(y), int(x)
                    r = int(s * 2)  # Blob radius

                    # Create circular region
                    y_min, y_max = max(0, y-r), min(inverted.shape[0], y+r+1)
                    x_min, x_max = max(0, x-r), min(inverted.shape[1], x+r+1)

                    # For each point in the potential circle
                    for yi in range(y_min, y_max):
                        for xi in range(x_min, x_max):
                            # Check if point is within radius
                            if ((yi-y)**2 + (xi-x)**2) <= r**2:
                                # Higher weight for larger blobs
                                temp_response[yi, xi] = max(temp_response[yi, xi], s/max(scales))

                # Add to blob response
                blob_response = np.maximum(blob_response, temp_response)

    # Normalize blob response
    if np.max(blob_response) > 0:
        blob_response = blob_response / np.max(blob_response)

    # Create output - holes should be dark in final image
    output = np.max(stack, axis=0)

    # Darken regions with high blob response
    output = output * (1.0 - 0.8 * blob_response)

    # Enhance contrast
    output = exposure.rescale_intensity(output)

    return output

def boundary_continuity_projection(image_stack, sigma=1.5, threshold=0.2):
    """
    Analyzes the continuity of cell boundaries to identify holes surrounded by
    continuous membrane staining, which is characteristic of true biological holes
    rather than imaging artifacts.
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize outputs
    edge_continuity = np.zeros_like(stack[0])
    enclosed_regions = np.zeros_like(stack[0], dtype=bool)

    # Process each z-slice
    for z in range(stack.shape[0]):
        # Detect edges
        edges = filters.sobel(filters.gaussian(stack[z], sigma=sigma)) > threshold

        # Thin edges to single pixel width
        edges = morphology.skeletonize(edges)

        # Find enclosed regions (potential holes)
        # Fill holes in the edge map
        filled_edges = ndimage.binary_fill_holes(edges)

        # Holes are regions that are filled but weren't edges
        potential_holes = filled_edges & ~edges

        # Remove small holes
        potential_holes = morphology.remove_small_objects(potential_holes, min_size=10)

        # Update enclosed regions
        enclosed_regions |= potential_holes

    # Create output
    output = np.max(stack, axis=0)

    # Darken enclosed regions (potential holes)
    output[enclosed_regions] = output[enclosed_regions] * 0.3

    # Enhance contrast
    output = exposure.rescale_intensity(output)

    return output

def wavelet_decomposition_projection(image_stack, level=3):
    """
    Uses wavelet decomposition to separate structural features at different scales,
    then reconstructs an image that emphasizes holes by manipulating the wavelet coefficients.
    """
    try:
        import pywt

        # Convert to float for processing
        stack = img_as_float(image_stack)

        # Initialize output
        wavelet_output = np.zeros_like(stack[0])

        # Process each z-slice
        for z in range(stack.shape[0]):
            try:
                # Apply wavelet decomposition
                coeffs = pywt.wavedec2(stack[z], 'sym2', level=level)

                # Modify coefficients to enhance holes
                # This method boosts mid-frequency components that often correspond to holes
                # while suppressing high-frequency noise and low-frequency background

                # Copy coefficients for manipulation
                modified_coeffs = list(coeffs)

                # Manipulate approximation coefficients (lowest frequency)
                # Reducing these reduces background influence
                modified_coeffs[0] *= 0.7

                # Enhance detail coefficients at mid-levels
                # These often correspond to structures like holes
                for i in range(1, min(3, len(modified_coeffs))):
                    # Each detail level has horizontal, vertical, and diagonal details
                    h, v, d = modified_coeffs[i]
                    modified_coeffs[i] = (h*1.5, v*1.5, d*1.5)

                # Reconstruct image from modified coefficients
                reconstructed = pywt.waverec2(modified_coeffs, 'sym2')

                # Handle edge effects by cropping if necessary
                if reconstructed.shape != stack[z].shape:
                    # Crop or pad to match original size
                    h_diff = reconstructed.shape[0] - stack[z].shape[0]
                    w_diff = reconstructed.shape[1] - stack[z].shape[1]

                    if h_diff > 0:
                        reconstructed = reconstructed[:stack[z].shape[0], :]
                    if w_diff > 0:
                        reconstructed = reconstructed[:, :stack[z].shape[1]]

                # Combine with output - taking the minimum will emphasize dark holes
                if z == 0:
                    wavelet_output = reconstructed
                else:
                    # Take minimum for dark regions (holes)
                    wavelet_output = np.minimum(wavelet_output, reconstructed)
            except Exception as e:
                print(f"Error in wavelet processing for slice {z}: {e}")
                # If wavelet fails, use a simple projection instead
                if z == 0:
                    wavelet_output = stack[z]
                else:
                    wavelet_output = np.minimum(wavelet_output, stack[z])

        # Normalize output
        wavelet_output = exposure.rescale_intensity(wavelet_output)

        return wavelet_output

    except ImportError:
        print("PyWavelets package not found. Using differential_density_projection instead.")
        return differential_density_projection(image_stack)

def dynamic_threshold_saliency_projection(image_stack, window_size=15, alpha=0.5):
    """
    Detects holes using a dynamic thresholding approach that adapts to local contrast
    and combines the results with visual saliency to highlight perceptually significant holes.
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize outputs
    hole_map = np.zeros_like(stack[0])

    # Process each z-slice
    for z in range(stack.shape[0]):
        # Calculate local mean and standard deviation
        local_mean = ndimage.uniform_filter(stack[z], size=window_size)
        local_std = np.sqrt(ndimage.uniform_filter(stack[z]**2, size=window_size) - local_mean**2)

        # Dynamic threshold based on local statistics
        # Pixels darker than (mean - k*std) are potential holes
        k = 1.5  # Threshold factor
        dynamic_thresh = local_mean - k * local_std

        # Identify potential holes
        potential_holes = stack[z] < dynamic_thresh

        # Clean up with morphological operations
        potential_holes = morphology.remove_small_objects(potential_holes, min_size=10)
        potential_holes = morphology.remove_small_holes(potential_holes, area_threshold=5)

        # Calculate visual saliency
        # Simple saliency based on contrast difference from surroundings
        saliency = np.abs(stack[z] - local_mean)
        saliency = filters.gaussian(saliency, sigma=2)

        # Normalize saliency
        if np.max(saliency) > 0:
            saliency = saliency / np.max(saliency)

        # Combine hole detection with saliency
        hole_score = potential_holes.astype(float) * (1.0 - alpha + alpha * saliency)

        # Update hole map - taking maximum across z-slices
        hole_map = np.maximum(hole_map, hole_score)

    # Create output
    output = np.max(stack, axis=0)

    # Enhance visualization by darkening detected holes
    output = output * (1.0 - 0.7 * hole_map)

    # Enhance contrast
    output = exposure.rescale_intensity(output)

    return output

def depth_coherence_projection(image_stack, coherence_threshold=0.6, min_depth=3):
    """
    Detects holes that maintain coherence across multiple z-slices,
    filtering out noise and artifacts that don't persist through depth.
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Skip if not enough z-slices
    if stack.shape[0] < min_depth:
        return np.max(stack, axis=0)

    # Initialize outputs
    coherent_holes = np.zeros_like(stack[0], dtype=bool)

    # Detect potential holes in each slice
    hole_candidates = []
    for z in range(stack.shape[0]):
        # Threshold to get dark regions (potential holes)
        thresh = filters.threshold_otsu(stack[z])
        dark_regions = stack[z] < thresh * 0.7

        # Clean up
        dark_regions = morphology.remove_small_objects(dark_regions, min_size=10)
        dark_regions = morphology.remove_small_holes(dark_regions, area_threshold=5)

        # Store
        hole_candidates.append(dark_regions)

    # Now check for coherence across depth
    depth_count = np.zeros_like(stack[0], dtype=int)
    for z in range(stack.shape[0] - min_depth + 1):
        # Look for consistent holes across min_depth consecutive slices
        consistent = hole_candidates[z].copy()
        for d in range(1, min_depth):
            consistent &= hole_candidates[z + d]

        # Count consistent depth locations
        depth_count[consistent] += min_depth

    # Identify regions with good depth coherence
    coherent_holes = depth_count >= (coherence_threshold * stack.shape[0])

    # Create output
    output = np.max(stack, axis=0)

    # Enhance visualization by darkening coherent holes
    output[coherent_holes] = output[coherent_holes] * 0.3

    # Enhance borders for better visibility
    hole_borders = morphology.dilation(coherent_holes, morphology.disk(1)) & ~coherent_holes
    output[hole_borders] = np.minimum(output[hole_borders] * 1.5, 1.0)

    # Enhance contrast
    output = exposure.rescale_intensity(output)

    return output

def constrained_active_contour_projection(image_stack, alpha=0.1, beta=0.2, gamma=0.01):
    """
    Projects holes detected using active contour models (snakes) that are
    constrained to focus on closed concave regions characteristic of cellular holes.
    """
    try:
        from skimage.segmentation import active_contour

        # Convert to float for processing
        stack = img_as_float(image_stack)

        # Initialize output
        output = np.max(stack, axis=0)
        detected_holes = np.zeros_like(output, dtype=bool)

        # Process each z-slice to find potential holes
        for z in range(stack.shape[0]):
            # Smooth the image
            smoothed = filters.gaussian(stack[z], sigma=1.0)

            # Get edges
            edges = filters.sobel(smoothed)

            # Threshold to get initial regions
            thresh = filters.threshold_otsu(smoothed)
            binary = smoothed < thresh * 0.8

            # Clean up
            binary = morphology.remove_small_objects(binary, min_size=20)
            binary = morphology.remove_small_holes(binary, area_threshold=10)

            # Label regions
            labeled = measure.label(binary)

            # Process each region as a potential hole
            for region in measure.regionprops(labeled):
                if 20 < region.area < 500:  # Size constraint
                    # Get region bounding box
                    minr, minc, maxr, maxc = region.bbox

                    # Pad to ensure contour has room to evolve
                    pad = 5
                    minr = max(0, minr - pad)
                    minc = max(0, minc - pad)
                    maxr = min(smoothed.shape[0], maxr + pad)
                    maxc = min(smoothed.shape[1], maxc + pad)

                    # Skip if region is too close to image boundary
                    if minr == 0 or minc == 0 or maxr == smoothed.shape[0] or maxc == smoothed.shape[1]:
                        continue

                    # Initial contour as a circle
                    center_r = (minr + maxr) // 2
                    center_c = (minc + maxc) // 2
                    radius = min(maxr - minr, maxc - minc) // 4

                    # Create circular contour
                    t = np.linspace(0, 2*np.pi, 100)
                    circle_r = center_r + radius * np.sin(t)
                    circle_c = center_c + radius * np.cos(t)
                    init_contour = np.column_stack([circle_c, circle_r])

                    # Create energy field for active contour
                    # Invert so holes (dark regions) attract the contour
                    energy_field = 1 - edges[minr:maxr, minc:maxc]

                    try:
                        # Evolve contour
                        snake = active_contour(
                            energy_field,
                            init_contour - [minc, minr],  # Adjust for ROI offset
                            alpha=alpha,
                            beta=beta,
                            gamma=gamma,
                            max_iterations=100
                        )

                        # Create mask from contour
                        snake_mask = np.zeros_like(energy_field, dtype=bool)

                        # Convert snake back to original coordinates
                        snake_coords = np.round(snake + [minc, minr]).astype(int)

                        # Create polygon mask from contour
                        from skimage.draw import polygon
                        rr, cc = polygon(snake_coords[:, 1], snake_coords[:, 0], energy_field.shape)

                        # Ensure coordinates are within image bounds
                        valid_pts = (
                            (rr >= 0) & (rr < energy_field.shape[0]) &
                            (cc >= 0) & (cc < energy_field.shape[1])
                        )
                        if np.any(valid_pts):
                            snake_mask[rr[valid_pts], cc[valid_pts]] = True

                        # Update detected holes in global coordinates
                        detected_holes[minr:maxr, minc:maxc][snake_mask] = True

                    except Exception as e:
                        # Skip problematic regions
                        continue

        # Create final output
        # Darken detected holes
        output[detected_holes] = output[detected_holes] * 0.3

        # Enhance borders for better visibility
        hole_borders = morphology.dilation(detected_holes, morphology.disk(1)) & ~detected_holes
        output[hole_borders] = np.minimum(output[hole_borders] * 1.5, 1.0)

        # Enhance contrast
        output = exposure.rescale_intensity(output)

        return output

    except ImportError:
        print("active_contour not available. Using differential_density_projection instead.")
        return differential_density_projection(image_stack)


# Dictionary of all projection methods
def get_all_projection_methods():
    """
    Returns a dictionary of all projection methods, including original and new ones
    """
    methods = {
        # Original methods
        'inverse_weighted': inverse_weighted_projection,
        'dark_spot': dark_spot_enhancement_projection,
        'junction_gap': junction_gap_projection,
        'hollow_structure': hollow_structure_projection,
        'diff_density': differential_density_projection,
        'phase_contrast': phase_contrast_simulation_projection,
        'local_minima': local_minima_projection,
        'vesicle_enhance': vesicle_enhancing_projection,

        # New advanced methods
        'texture_contrast': texture_contrast_projection,
        'directional_gradient': directional_gradient_flow_projection,
        'multiscale_hole': multiscale_hole_detection_projection,
        'boundary_continuity': boundary_continuity_projection,
        'wavelet_decomp': wavelet_decomposition_projection,
        'dynamic_threshold': dynamic_threshold_saliency_projection,
        'depth_coherence': depth_coherence_projection,
        'active_contour': constrained_active_contour_projection
    }

    return methods


def process_4d_microscopy_image(image, filename, process_first_channel_only=True):
    """
    Process a 4D microscopy image with new projection methods for vacuole detection

    Parameters:
    -----------
    image : numpy.ndarray
        4D input image with shape (channels, z-slices, height, width)
    filename : str
        Original filename for naming the output
    process_first_channel_only : bool
        If True, only process the first channel

    Returns:
    --------
    dict
        Dictionary of processed images
    """
    print(f"  Processing 4D image with shape {image.shape}")

    # Normalize values if needed
    if image.dtype == np.float32 or image.dtype == np.float64:
        if np.max(image) > 1.0 or np.min(image) < -1.0:
            print(f"  Normalizing image values from range [{np.min(image)}, {np.max(image)}] to [0, 1]")
            for c in range(image.shape[0]):
                for z in range(image.shape[1]):
                    image[c, z] = normalize_image(image[c, z])

    projections = {}

    # Determine which channels to process
    channels_to_process = [0] if process_first_channel_only else range(image.shape[0])

    # Process each channel
    for channel in channels_to_process:
        # Get all z-slices for this channel
        channel_data = image[channel]

        # Get all projection methods
        vacuole_detection_methods = get_all_projection_methods()

        # Apply each method and enhance result
        for method_name, method_func in vacuole_detection_methods.items():
            try:
                # Apply projection method
                print(f"  Applying {method_name} projection to channel {channel+1}...")
                proj = method_func(channel_data)

                # Apply CLAHE for better visualization
                enhanced = exposure.equalize_adapthist(proj, clip_limit=0.03)

                # Store result
                key = f"channel_{channel+1}_{method_name}"
                projections[key] = enhanced
                print(f"  Successfully created {key} projection")
            except Exception as e:
                print(f"  Error applying {method_name} projection: {e}")
                traceback.print_exc()

    return projections


def process_all_images():
    """
    Process all 4D microscopy images in the input directory
    """
    # Get all image files with common extensions (case insensitive)
    extensions = ['.tif', '.tiff', '.TIF', '.TIFF']
    all_files = []

    for ext in extensions:
        all_files.extend(glob.glob(os.path.join(input_dir, f'*{ext}')))

    if len(all_files) == 0:
        print(f"No image files found in {input_dir}")
        return []

    # Sort files to ensure consistent processing order
    all_files.sort()

    print(f"Found {len(all_files)} image files. Processing each with multiple methods...")

    processed_files = []

    # Process each image
    for idx, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        print(f"Processing image {idx+1}/{len(all_files)}: {filename}")

        try:
            # Load the image
            image = tifffile.imread(file_path)

            # Print image info
            print(f"  Image shape: {image.shape}, dtype: {image.dtype}")

            # Skip if not 4D
            if len(image.shape) != 4:
                print(f"  Skipping - not a 4D image")
                continue

            # Process the 4D image (only first channel)
            projections = process_4d_microscopy_image(image, filename, process_first_channel_only=True)

            # Save each projection
            for proj_name, projection in projections.items():
                # Create output filename
                base_name = os.path.splitext(filename)[0]
                output_filename = f"{base_name}_{proj_name}.tif"
                output_path = os.path.join(output_dir, output_filename)

                # Convert to uint16 for saving
                if projection.dtype == np.float64 or projection.dtype == np.float32:
                    projection = img_as_uint(projection)

                # Save the projection
                tifffile.imwrite(output_path, projection)

                # Add to list of processed files
                processed_files.append(output_path)

            print(f"  Saved {len(projections)} projections")

        except Exception as e:
            print(f"  Error processing image {filename}: {e}")
            traceback.print_exc()
            continue

    return processed_files


# Main execution
if __name__ == "__main__":
    print("Starting 4D image processing with advanced vacuole detection projection methods...")
    print(f"Input directory: {input_dir}")
    print(f"Output directory: {output_dir}")

    # Process all image files in the directory
    processed_files = process_all_images()

    # Display results
    if processed_files:
        print(f"\nSuccessfully processed {len(processed_files)} projections:")
        for output_file in processed_files[:5]:  # Show first 5
            print(f"  - {os.path.basename(output_file)}")
        if len(processed_files) > 5:
            print(f"  ... and {len(processed_files) - 5} more")

        # Try to display the first result in the notebook
        try:
            from IPython.display import Image
            print("\nDisplaying first result:")
            Image(processed_files[0])
        except Exception as e:
            print(f"Couldn't display image: {e}")
    else:
        print("No images were processed successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting 4D image processing with advanced vacuole detection projection methods...
Input directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial
Output directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/trial23
Found 1 image files. Processing each with multiple methods...
Processing image 1/1: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1).tif
  Image shape: (3, 21, 1024, 1024), dtype: float32
  Processing 4D image with shape (3, 21, 1024, 1024)
  Normalizing image values from range [-0.02250508777797222, 3.4323275089263916] to [0, 1]
  Applying inverse_weighted projection to channel 1...
  Successfully created channel_1_inverse_weighted projection
  Applying dark_spot projection to channel 1...
  Successfully created channel_1_dark_spot projection
  Applying junction_gap projection to channel 1...


Traceback (most recent call last):
  File "<ipython-input-7-b82d38ead205>", line 974, in process_4d_microscopy_image
    proj = method_func(channel_data)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-7-b82d38ead205>", line 270, in local_minima_projection
    local_min = feature.peak_local_max(
                ^^^^^^^^^^^^^^^^^^^^^^^
TypeError: peak_local_max() got an unexpected keyword argument 'indices'


  Successfully created channel_1_vesicle_enhance projection
  Applying texture_contrast projection to channel 1...


/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


  Successfully created channel_1_texture_contrast projection
  Applying directional_gradient projection to channel 1...
  Successfully created channel_1_directional_gradient projection
  Applying multiscale_hole projection to channel 1...
  Successfully created channel_1_multiscale_hole projection
  Applying boundary_continuity projection to channel 1...
  Successfully created channel_1_boundary_continuity projection
  Applying wavelet_decomp projection to channel 1...
PyWavelets package not found. Using differential_density_projection instead.
  Successfully created channel_1_wavelet_decomp projection
  Applying dynamic_threshold projection to channel 1...
  Successfully created channel_1_dynamic_threshold projection
  Applying depth_coherence projection to channel 1...
  Successfully created channel_1_depth_coherence projection
  Applying active_contour projection to channel 1...
  Successfully created channel_1_active_contour projection
  Saved 15 projections

Successfully processed

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
from skimage import filters, morphology, feature, measure, exposure, img_as_float, img_as_uint, segmentation, color, transform
from scipy import ndimage
import matplotlib.pyplot as plt
import glob
import tifffile
import cv2

# Define input and output paths (same as original)
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/trial24'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def normalize_image(img):
    """Normalize image to 0-1 range"""
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:
        return (img - img_min) / (img_max - img_min)
    return img

# Original projection methods from your code

def inverse_weighted_projection(image_stack, sigma=1.5):
    """
    Inverse-weighted projection that emphasizes dark regions (potential vacuoles)
    by giving higher weight to darker voxels in the z-direction
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Invert the stack to emphasize dark regions
    inverted_stack = 1 - stack

    # Calculate weights based on darkness (higher value = darker pixel)
    weights = filters.gaussian(inverted_stack, sigma=sigma)

    # Normalize weights for each pixel position
    weight_sum = np.sum(weights, axis=0)
    weight_sum[weight_sum == 0] = 1  # Avoid division by zero

    # Calculate weighted average of original (non-inverted) stack
    output = np.zeros_like(stack[0])
    for i in range(stack.shape[0]):
        output += stack[i] * weights[i] / weight_sum

    return output

def dark_spot_enhancement_projection(image_stack, spot_size=5):
    """
    Projects the stack while enhancing dark spots of a specific size range
    Useful for highlighting vacuoles/holes in cadherin-stained cells
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create output with maximum values first
    output = np.max(stack, axis=0)

    # Process each z-slice to detect dark spots
    for z in range(stack.shape[0]):
        # Detect dark spots using white top-hat transform on inverted image
        inverted = 1 - stack[z]
        selem = morphology.disk(spot_size)
        dark_spots = morphology.white_tophat(inverted, selem)

        # Threshold to identify significant dark spots
        threshold = filters.threshold_otsu(dark_spots) * 0.5
        dark_spots_binary = dark_spots > threshold

        # Label connected regions
        labeled_spots = measure.label(dark_spots_binary)

        # For each labeled region, check if it's a potential vacuole
        for region in measure.regionprops(labeled_spots):
            # Filter based on size and shape
            if region.area > 10 and region.area < 500 and region.eccentricity < 0.8:
                # Get region coordinates
                coords = region.coords

                # Update output image to show this dark spot
                for coord in coords:
                    y, x = coord
                    # Replace with dark value from this slice if it's darker
                    if stack[z, y, x] < output[y, x]:
                        output[y, x] = stack[z, y, x]

    return output

def junction_gap_projection(image_stack, membrane_thickness=3):
    """
    Specifically designed to detect gaps in cadherin staining at cell junctions
    by looking for discontinuities in membrane staining patterns
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Get maximum intensity projection as base
    mip = np.max(stack, axis=0)

    # Enhance edges to highlight cell boundaries
    edges = np.zeros_like(mip)
    for z in range(stack.shape[0]):
        # Detect edges using Sobel filter
        edge_sobel = filters.sobel(stack[z])
        edges = np.maximum(edges, edge_sobel)

    # Threshold edges to get membrane mask
    edge_threshold = filters.threshold_otsu(edges) * 0.5
    membrane_mask = edges > edge_threshold

    # Dilate to connect nearby membrane segments
    membrane_mask = morphology.binary_dilation(membrane_mask, morphology.disk(membrane_thickness))

    # Find potential gaps - areas where membrane should be but intensity is low
    gap_candidates = np.zeros_like(mip)

    # Process each z-slice to look for gaps in membranes
    for z in range(stack.shape[0]):
        # Get regions where membrane mask exists but pixel intensity is low
        low_intensity = stack[z] < np.percentile(stack[z][membrane_mask], 30)
        potential_gaps = membrane_mask & low_intensity

        # Add to candidates
        gap_candidates[potential_gaps] += 1

    # Normalize gap candidates
    gap_candidates = gap_candidates / stack.shape[0]

    # Create output that emphasizes gaps in junctions
    output = mip.copy()

    # Highlight gap regions by reducing intensity
    gap_regions = gap_candidates > 0.5
    output[gap_regions] = output[gap_regions] * 0.5

    return output

def hollow_structure_projection(image_stack, threshold=0.2, min_size=20, max_size=500):
    """
    Projects 3D hollow structures (potential vacuoles) by identifying dark regions
    surrounded by brighter regions across multiple z-slices
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Base projection - maximum intensity
    output = np.max(stack, axis=0)

    # Store potential hollow structures
    hollow_mask = np.zeros_like(output, dtype=bool)

    # Process each z slice
    for z in range(1, stack.shape[0]-1):  # Skip first and last slice
        # Threshold this slice to get foreground
        binary = stack[z] > threshold

        # Find holes in the binary image
        holes = morphology.remove_small_holes(binary, min_size) != binary

        # Remove small objects
        holes = morphology.remove_small_objects(holes, min_size)

        # Label connected components
        labeled_holes = measure.label(holes)

        # For each hole, check if it's surrounded by higher intensity in z-direction
        for region in measure.regionprops(labeled_holes):
            if region.area < max_size:  # Size constraint
                # Get region mask
                y0, x0, y1, x1 = region.bbox
                region_mask = labeled_holes[y0:y1, x0:x1] == region.label

                # Check intensity above and below
                above_intensity = np.mean(stack[min(z+1, stack.shape[0]-1), y0:y1, x0:x1][region_mask])
                below_intensity = np.mean(stack[max(z-1, 0), y0:y1, x0:x1][region_mask])
                current_intensity = np.mean(stack[z, y0:y1, x0:x1][region_mask])

                # If current slice is darker than slices above and below, it might be a hollow structure
                if current_intensity < above_intensity*0.8 and current_intensity < below_intensity*0.8:
                    hollow_mask[y0:y1, x0:x1] |= region_mask

    # Enhance the visualization by making hollow regions darker
    output[hollow_mask] = output[hollow_mask] * 0.3

    return output

def differential_density_projection(image_stack, kernel_size=5):
    """
    Highlights regions where there's significant difference in density/intensity
    between adjacent z-layers, useful for detecting vacuole boundaries
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Calculate gradient magnitude in z direction
    z_gradient = np.zeros_like(stack)
    for z in range(1, stack.shape[0]):
        z_gradient[z] = np.abs(stack[z] - stack[z-1])

    # Smooth the gradient
    smoothed_gradient = np.zeros_like(z_gradient)
    for z in range(z_gradient.shape[0]):
        smoothed_gradient[z] = filters.gaussian(z_gradient[z], sigma=1)

    # Get the maximum gradient projection
    max_gradient = np.max(smoothed_gradient, axis=0)

    # Create output that combines intensity and gradient information
    # Higher gradient indicates boundary of structure
    output = np.max(stack, axis=0)

    # Enhance regions with high gradient
    norm_gradient = normalize_image(max_gradient)
    output = output * (1 - norm_gradient * 0.5)

    return output

def phase_contrast_simulation_projection(image_stack, sigma=2):
    """
    Simulates phase contrast-like visualization to enhance membrane and vacuole boundaries
    by emphasizing intensity gradients in all directions
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Create output image
    output = np.zeros_like(stack[0])

    # Process each z slice
    for z in range(stack.shape[0]):
        # Apply Laplacian of Gaussian filter to simulate phase contrast
        log_filtered = filters.laplace(filters.gaussian(stack[z], sigma=sigma))

        # Normalize to -1 to 1 range
        log_filtered = 2 * normalize_image(log_filtered) - 1

        # Convert to phase contrast-like appearance
        phase_contrast = 0.5 + 0.5 * log_filtered

        # Add to output based on original intensity (weighted)
        output += phase_contrast * stack[z]

    # Normalize
    output = normalize_image(output)

    return output

def local_minima_projection(image_stack, footprint_size=5):
    """
    Projects local minima across the z-stack, which could correspond to vacuoles
    or gaps in cadherin staining
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize with maximum values
    output = np.ones_like(stack[0])

    # Process each z slice
    for z in range(stack.shape[0]):
        # Find local minima
        footprint = morphology.disk(footprint_size)
        local_min = feature.peak_local_max(
            -stack[z],  # Invert to find minima
            footprint=footprint,
            indices=False
        )

        # For each local minimum, check if it's a potential vacuole
        labeled_mins = measure.label(local_min)

        for region in measure.regionprops(labeled_mins):
            y, x = region.centroid
            y, x = int(y), int(x)

            # Update output with local minimum value if it's lower
            if stack[z, y, x] < output[y, x]:
                # Update a small region around the minimum
                y_min, x_min = max(0, y-2), max(0, x-2)
                y_max, x_max = min(stack.shape[1], y+3), min(stack.shape[2], x+3)

                # Check each pixel in the neighborhood
                for yi in range(y_min, y_max):
                    for xi in range(x_min, x_max):
                        if stack[z, yi, xi] < output[yi, xi]:
                            output[yi, xi] = stack[z, yi, xi]

    return output

def vesicle_enhancing_projection(image_stack, min_size=5, max_size=50):
    """
    Specifically designed to enhance small vesicles or vacuoles
    by detecting spherical hollow structures across z-slices
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize outputs
    output = np.max(stack, axis=0)  # Base is maximum projection
    vesicle_mask = np.zeros_like(output, dtype=bool)

    # Step 1: Detect circular structures in each slice
    for z in range(stack.shape[0]):
        # Apply a white tophat filter to enhance small structures
        tophat = morphology.white_tophat(1-stack[z], morphology.disk(max_size//2))

        # Threshold to get potential vesicles
        thresh = filters.threshold_otsu(tophat) * 0.7
        binary = tophat > thresh

        # Remove small noise and fill holes
        binary = morphology.remove_small_objects(binary, min_size)
        binary = morphology.remove_small_holes(binary)

        # Label connected components
        labeled = measure.label(binary)

        # Analyze each region
        for region in measure.regionprops(labeled):
            # Filter based on size and circularity
            if (min_size < region.area < max_size and
                region.eccentricity < 0.6 and  # Circular shape
                region.solidity > 0.8):        # Solid/compact

                # Get coordinates
                y0, x0, y1, x1 = region.bbox
                mask = labeled[y0:y1, x0:x1] == region.label

                # Check intensity inside vs outside
                inside_intensity = np.mean(stack[z, y0:y1, x0:x1][mask])

                # Create dilated mask for outside region
                dilated_mask = morphology.binary_dilation(mask, morphology.disk(2))
                dilated_mask = dilated_mask & ~mask

                # Check if there's enough border pixels
                if np.sum(dilated_mask) > 0:
                    outside_intensity = np.mean(stack[z, y0:y1, x0:x1][dilated_mask])

                    # If inside is significantly darker than outside, it's a potential vesicle
                    if inside_intensity < outside_intensity * 0.7:
                        vesicle_mask[y0:y1, x0:x1] |= mask

    # Enhance visualization by darkening detected vesicles
    output[vesicle_mask] = output[vesicle_mask] * 0.2

    # And brightening their boundaries for better visibility
    vesicle_boundary = morphology.binary_dilation(vesicle_mask) & ~vesicle_mask
    output[vesicle_boundary] = np.minimum(output[vesicle_boundary] * 1.5, 1.0)

    return output

# New innovative projection methods

def texture_discontinuity_projection(image_stack, patch_size=7, distance_metric='correlation'):
    """
    Projects regions with texture discontinuities which may represent holes in cell monolayers.
    Uses local texture features to identify areas that differ from their surroundings.

    Parameters:
    -----------
    image_stack : numpy.ndarray
        3D stack of images (z, height, width)
    patch_size : int
        Size of patches to compare textures
    distance_metric : str
        Metric to compare textures ('correlation', 'chi-square', 'hellinger')

    Returns:
    --------
    numpy.ndarray
        2D projection highlighting texture discontinuities
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Output will be a weighted projection
    weights = np.zeros_like(stack)

    # For each z-slice, calculate local texture properties
    for z in range(stack.shape[0]):
        # Calculate local texture features using Gabor filters
        img = stack[z]

        # Apply Gabor filters at different orientations to capture textures
        texture_map = np.zeros_like(img)

        for theta in [0, np.pi/4, np.pi/2, 3*np.pi/4]:
            # Create Gabor kernel
            kernel = cv2.getGaborKernel(
                (patch_size, patch_size),
                sigma=2.0,
                theta=theta,
                lambd=10.0,
                gamma=0.5,
                psi=0,
                ktype=cv2.CV_32F
            )

            # Apply filter
            filtered = cv2.filter2D(img.astype(np.float32), cv2.CV_32F, kernel)
            texture_map += np.abs(filtered)

        # Normalize texture map
        texture_map = texture_map / (texture_map.max() + 1e-10)

        # Calculate local variance in textures using sliding window
        texture_variance = ndimage.generic_filter(
            texture_map, np.var, size=patch_size
        )

        # Normalize variance
        texture_variance = texture_variance / (texture_variance.max() + 1e-10)

        # Higher variance indicates discontinuities - potential holes
        weights[z] = texture_variance

    # Create weighted projection
    # Higher weight for slices with more texture discontinuities
    weight_sum = np.sum(weights, axis=0)
    weight_sum[weight_sum == 0] = 1  # Avoid division by zero

    # Weighted projection - we actually want the original image
    output = np.zeros_like(stack[0])
    for z in range(stack.shape[0]):
        output += stack[z] * weights[z] / weight_sum

    # Enhance the contrast of the discontinuities
    output = exposure.rescale_intensity(output)

    return output

def adaptive_multi_threshold_projection(image_stack, num_thresholds=3, hole_size_range=(20, 500)):
    """
    Uses multiple adaptive thresholds to identify potential holes across different intensity ranges.
    Combines results to create a comprehensive projection.

    Parameters:
    -----------
    image_stack : numpy.ndarray
        3D stack of images (z, height, width)
    num_thresholds : int
        Number of thresholds to use
    hole_size_range : tuple
        (min_size, max_size) for hole candidates

    Returns:
    --------
    numpy.ndarray
        2D projection highlighting potential holes
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Get max projection as base
    max_proj = np.max(stack, axis=0)
    output = max_proj.copy()

    # Accumulator for hole candidates
    hole_candidates = np.zeros_like(max_proj, dtype=float)

    # For each z-slice, apply multiple adaptive thresholds
    for z in range(stack.shape[0]):
        img = stack[z]

        # Apply multiple thresholds
        percentiles = np.linspace(20, 80, num_thresholds)

        for p in percentiles:
            # Adaptive threshold using percentile
            threshold = np.percentile(img, p)
            binary = img < threshold

            # Clean up binary image
            binary = morphology.remove_small_objects(binary, min_size=hole_size_range[0])
            binary = morphology.remove_small_holes(binary, area_threshold=hole_size_range[1])

            # Label connected components
            labeled = measure.label(binary)

            # Analyze each region
            for region in measure.regionprops(labeled):
                # Filter by size
                if hole_size_range[0] <= region.area <= hole_size_range[1]:
                    # Get region mask
                    y0, x0, y1, x1 = region.bbox
                    mask = labeled[y0:y1, x0:x1] == region.label

                    # Check if the region is darker than surroundings
                    inside = img[y0:y1, x0:x1][mask].mean()

                    # Create dilated mask for outside
                    dilated = morphology.binary_dilation(mask, morphology.disk(3))
                    outside_mask = dilated & ~mask

                    # Only check outside if there are pixels there
                    if np.any(outside_mask):
                        outside = img[y0:y1, x0:x1][outside_mask].mean()

                        # Ratio of inside vs outside intensity
                        ratio = inside / (outside + 1e-10)

                        # If inside is darker than outside, count as hole candidate
                        if ratio < 0.8:
                            # Add to candidates with a score based on contrast
                            score = 1 - ratio  # Higher score for greater contrast
                            hole_candidates[y0:y1, x0:x1][mask] += score

    # Normalize hole candidates
    if np.max(hole_candidates) > 0:
        hole_candidates = hole_candidates / np.max(hole_candidates)

    # Apply hole candidates to output
    # Enhance darkness of hole regions proportional to their score
    output = output * (1 - 0.8 * hole_candidates)

    return output

def wavelet_decomposition_projection(image_stack, level=3):
    """
    Uses wavelet decomposition to separate different scales of features,
    then reconstructs with emphasis on scales likely to contain holes.

    Parameters:
    -----------
    image_stack : numpy.ndarray
        3D stack of images (z, height, width)
    level : int
        Decomposition level

    Returns:
    --------
    numpy.ndarray
        2D projection highlighting potential holes
    """
    try:
        import pywt
        pywt_available = True
    except ImportError:
        print("PyWavelets package not found. Install with: pip install PyWavelets")
        pywt_available = False

    # Fallback to a simpler method if PyWavelets not available
    if not pywt_available:
        return differential_density_projection(image_stack)

    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Output will accumulate wavelet-enhanced features
    output = np.zeros_like(stack[0])

    for z in range(stack.shape[0]):
        img = stack[z]

        # Wavelet decomposition
        coeffs = pywt.wavedec2(img, 'db1', level=level)

        # Modify coefficients to enhance hole-like structures
        # The approximation coefficients (low frequency)
        cA = coeffs[0]

        # Detail coefficients (high frequency)
        detail_coeffs = list(coeffs[1:])

        # Enhance horizontal, vertical and diagonal details
        # at scales likely to contain holes
        for i in range(len(detail_coeffs)):
            # Determine weight based on level (medium scales usually contain holes)
            if i < level - 1:  # Smaller details, possibly noise
                weight = 0.5
            else:  # Larger details, possibly holes
                weight = 2.0

            # Apply weight to detail coefficients
            (cH, cV, cD) = detail_coeffs[i]
            detail_coeffs[i] = (cH * weight, cV * weight, cD * weight)

        # Reconstruct with modified coefficients
        modified_coeffs = [cA] + detail_coeffs
        reconstructed = pywt.waverec2(modified_coeffs, 'db1')

        # Need to ensure the reconstructed image has the same shape as the original
        # Sometimes waverec2 returns a slightly different shape
        if reconstructed.shape != img.shape:
            reconstructed = transform.resize(reconstructed, img.shape, mode='edge')

        # Add to output
        output += reconstructed

    # Normalize
    output = output / stack.shape[0]

    # Enhance contrast
    output = exposure.rescale_intensity(output)

    return output

def topological_watershed_projection(image_stack, sigma=1.5, h_param=0.05):
    """
    Uses the watershed algorithm with topological constraints to identify regions
    that are topologically consistent with holes across multiple z-slices.

    Parameters:
    -----------
    image_stack : numpy.ndarray
        3D stack of images (z, height, width)
    sigma : float
        Gaussian smoothing parameter
    h_param : float
        Height parameter for h-minima transform

    Returns:
    --------
    numpy.ndarray
        2D projection highlighting potential holes using watershed
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Get max projection as base
    max_proj = np.max(stack, axis=0)

    # Accumulator for watershed markers
    markers_accumulator = np.zeros_like(max_proj, dtype=int)

    # Process each z-slice
    for z in range(stack.shape[0]):
        # Get image and invert it (since we're looking for dark holes)
        img = stack[z]
        inverted = 1 - img

        # Smooth
        smoothed = filters.gaussian(inverted, sigma=sigma)

        # H-minima transform to find markers
        h_minima = morphology.h_minima(smoothed, h_param)

        # Label minima
        markers = measure.label(h_minima)

        # Calculate gradient magnitude as elevation map
        elevation = filters.sobel(smoothed)

        # Apply watershed
        watershed = segmentation.watershed(elevation, markers)

        # Add to accumulator
        markers_accumulator += watershed

    # Normalize accumulator
    markers_normalized = markers_accumulator / stack.shape[0]

    # Create output by modifying max projection
    output = max_proj.copy()

    # Find regions with consistent watershed labeling across z-slices
    # These are likely to be consistent structures (holes)
    consistent_regions = markers_normalized > 0.6  # Threshold for consistency

    # Make these regions darker in the output
    output[consistent_regions] = output[consistent_regions] * 0.5

    return output

def frequency_domain_projection(image_stack, radius_range=(3, 15), strength=1.5):
    """
    Uses frequency domain filtering to enhance structures of specific sizes.
    The size range can be tuned to target hole-like structures.

    Parameters:
    -----------
    image_stack : numpy.ndarray
        3D stack of images (z, height, width)
    radius_range : tuple
        (min_radius, max_radius) for bandpass filter in frequency domain
    strength : float
        Strength of the enhancement

    Returns:
    --------
    numpy.ndarray
        2D projection with frequency-domain enhanced features
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Initialize output with zeros
    output = np.zeros_like(stack[0])

    # Process each z-slice
    for z in range(stack.shape[0]):
        img = stack[z]

        # Apply FFT
        f_img = np.fft.fft2(img)
        f_img_shifted = np.fft.fftshift(f_img)

        # Create a bandpass filter mask
        rows, cols = img.shape
        crow, ccol = rows // 2, cols // 2

        # Initialize mask
        mask = np.zeros((rows, cols), dtype=np.float32)

        # Create bandpass filter
        for i in range(rows):
            for j in range(cols):
                # Calculate distance from center
                dist = np.sqrt((i - crow)**2 + (j - ccol)**2)

                # Bandpass filter
                if radius_range[0] <= dist <= radius_range[1]:
                    # Gaussian weighting within the band
                    weight = np.exp(-0.5 * ((dist - (radius_range[0] + radius_range[1])/2) /
                                          ((radius_range[1] - radius_range[0])/4))**2)
                    mask[i, j] = weight * strength

        # Apply filter
        f_img_filtered = f_img_shifted * mask

        # Inverse FFT
        img_back = np.fft.ifftshift(f_img_filtered)
        img_back = np.fft.ifft2(img_back)
        img_back = np.abs(img_back)

        # Normalize
        img_back = (img_back - np.min(img_back)) / (np.max(img_back) - np.min(img_back) + 1e-10)

        # Add to output
        output += img_back

    # Normalize output
    output = output / stack.shape[0]

    return output

# Helper function for circle drawing
def draw_circle(r, c, radius, shape):
    """Draw a circle at row r, column c with given radius within shape boundaries"""
    rr, cc = [], []
    for i in range(max(0, r - radius), min(shape[0], r + radius + 1)):
        for j in range(max(0, c - radius), min(shape[1], c + radius + 1)):
            if (i - r)**2 + (j - c)**2 <= radius**2:
                rr.append(i)
                cc.append(j)
    return np.array(rr), np.array(cc)

def multi_scale_blob_detection_projection(image_stack, min_sigma=1, max_sigma=5, num_sigma=5, threshold=0.02):
    """
    Uses multi-scale blob detection to find hole-like structures at different scales
    and creates a projection that emphasizes these structures.

    Parameters:
    -----------
    image_stack : numpy.ndarray
        3D stack of images (z, height, width)
    min_sigma : float
        Minimum scale for blob detection
    max_sigma : float
        Maximum scale for blob detection
    num_sigma : int
        Number of scales to use
    threshold : float
        Threshold for blob detection

    Returns:
    --------
    numpy.ndarray
        2D projection highlighting multi-scale blobs
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Max projection as base
    max_proj = np.max(stack, axis=0)
    output = max_proj.copy()

    # Accumulator for blob locations
    blob_map = np.zeros_like(max_proj)

    # Process each z-slice
    for z in range(stack.shape[0]):
        # Invert image since we're looking for dark blobs
        img = 1 - stack[z]

        # Detect blobs across multiple scales
        blobs = feature.blob_log(
            img,
            min_sigma=min_sigma,
            max_sigma=max_sigma,
            num_sigma=num_sigma,
            threshold=threshold
        )

        # If blobs were found
        if blobs.shape[0] > 0:
            # Add each blob to the map
            for blob in blobs:
                y, x, r = blob
                y, x = int(y), int(x)
                r = int(r * 1.5)  # Scale radius for visualization

                # Draw blob on map with intensity based on scale
                rr, cc = draw_circle(y, x, r, output.shape)
                blob_map[rr, cc] += 1.0 / (r + 1.0)  # Weight by inverse radius

    # Normalize blob map
    if np.max(blob_map) > 0:
        blob_map = blob_map / np.max(blob_map)

    # Apply blob map to output - make the blobs darker
    output = output * (1 - 0.7 * blob_map)

    return output

def adaptive_ensemble_projection(image_stack, method_weights=None):
    """
    Combines multiple projection methods with adaptive weighting
    based on image features to create a comprehensive visualization.

    Parameters:
    -----------
    image_stack : numpy.ndarray
        3D stack of images (z, height, width)
    method_weights : dict
        Optional weights for different methods

    Returns:
    --------
    numpy.ndarray
        2D projection created by adaptively combining multiple methods
    """
    # Convert to float for processing
    stack = img_as_float(image_stack)

    # Define methods to use in ensemble
    methods = [
        texture_discontinuity_projection,
        adaptive_multi_threshold_projection,
        frequency_domain_projection,
        topological_watershed_projection,
        multi_scale_blob_detection_projection
    ]

    # Try to include wavelet method if PyWavelets is available
    try:
        import pywt
        methods.append(wavelet_decomposition_projection)
    except ImportError:
        pass

    # Default weights if not specified
    if method_weights is None:
        method_weights = {
            texture_discontinuity_projection.__name__: 0.2,
            adaptive_multi_threshold_projection.__name__: 0.2,
            frequency_domain_projection.__name__: 0.15,
            topological_watershed_projection.__name__: 0.15,
            multi_scale_blob_detection_projection.__name__: 0.15,
            wavelet_decomposition_projection.__name__: 0.15
        }

    # Apply each method and collect projections
    projections = {}
    for method in methods:
        try:
            method_name = method.__name__
            print(f"  Applying {method_name} for ensemble...")
            proj = method(stack)
            projections[method_name] = proj
        except Exception as e:
            print(f"  Warning: Method {method.__name__} failed for ensemble: {e}")

    if not projections:
        print("  No projection methods succeeded for ensemble, using max projection")
        return np.max(stack, axis=0)

    # Combine projections using weights
    output = np.zeros_like(list(projections.values())[0])
    weight_sum = 0

    for method_name, proj in projections.items():
        weight = method_weights.get(method_name, 0.1)
        output += proj * weight
        weight_sum += weight

    # Normalize by total weight
    if weight_sum > 0:
        output = output / weight_sum

    return output

def process_4d_microscopy_image(image, filename, process_first_channel_only=True):
    """
    Process a 4D microscopy image with new projection methods for vacuole detection

    Parameters:
    -----------
    image : numpy.ndarray
        4D input image with shape (channels, z-slices, height, width)
    filename : str
        Original filename for naming the output
    process_first_channel_only : bool
        If True, only process the first channel

    Returns:
    --------
    dict
        Dictionary of processed images
    """
    print(f"  Processing 4D image with shape {image.shape}")

    # Normalize values if needed
    if image.dtype == np.float32 or image.dtype == np.float64:
        if np.max(image) > 1.0 or np.min(image) < -1.0:
            print(f"  Normalizing image values from range [{np.min(image)}, {np.max(image)}] to [0, 1]")
            for c in range(image.shape[0]):
                for z in range(image.shape[1]):
                    image[c, z] = normalize_image(image[c, z])

    projections = {}

    # Determine which channels to process
    channels_to_process = [0] if process_first_channel_only else range(image.shape[0])

    # Process each channel
    for channel in channels_to_process:
        # Get all z-slices for this channel
        channel_data = image[channel]

        # Original projection methods
        original_methods = {
            'inverse_weighted': inverse_weighted_projection,
            'dark_spot': dark_spot_enhancement_projection,
            'junction_gap': junction_gap_projection,
            'hollow_structure': hollow_structure_projection,
            'diff_density': differential_density_projection,
            'phase_contrast': phase_contrast_simulation_projection,
            'local_minima': local_minima_projection,
            'vesicle_enhance': vesicle_enhancing_projection
        }

        # New innovative projection methods
        innovative_methods = {
            'texture_discontinuity': texture_discontinuity_projection,
            'adaptive_multi_threshold': adaptive_multi_threshold_projection,
            'frequency_domain': frequency_domain_projection,
            'topological_watershed': topological_watershed_projection,
            'multi_scale_blob': multi_scale_blob_detection_projection,
            'adaptive_ensemble': adaptive_ensemble_projection
        }

        # Try to include wavelet if PyWavelets is available
        try:
            import pywt
            innovative_methods['wavelet'] = wavelet_decomposition_projection
        except ImportError:
            print("  PyWavelets not available. Skipping wavelet projection.")

        # Combine all methods
        all_methods = {**original_methods, **innovative_methods}

        # Apply each method and enhance result
        for method_name, method_func in all_methods.items():
            try:
                # Apply projection method
                print(f"  Applying {method_name} projection to channel {channel+1}...")
                proj = method_func(channel_data)

                # Apply CLAHE for better visualization
                enhanced = exposure.equalize_adapthist(proj, clip_limit=0.03)

                # Store result
                key = f"channel_{channel+1}_{method_name}"
                projections[key] = enhanced
                print(f"  Successfully created {key} projection")
            except Exception as e:
                print(f"  Error applying {method_name} projection: {e}")
                import traceback
                traceback.print_exc()

    return projections

def process_all_images():
    """
    Process all 4D microscopy images in the input directory
    """
    # Get all image files with common extensions (case insensitive)
    extensions = ['.tif', '.tiff', '.TIF', '.TIFF']
    all_files = []

    for ext in extensions:
        all_files.extend(glob.glob(os.path.join(input_dir, f'*{ext}')))

    if len(all_files) == 0:
        print(f"No image files found in {input_dir}")
        return []

    # Sort files to ensure consistent processing order
    all_files.sort()

    print(f"Found {len(all_files)} image files. Processing each with multiple methods...")

    processed_files = []

    # Process each image
    for idx, file_path in enumerate(all_files):
        filename = os.path.basename(file_path)
        print(f"Processing image {idx+1}/{len(all_files)}: {filename}")

        try:
            # Load the image
            image = tifffile.imread(file_path)

            # Print image info
            print(f"  Image shape: {image.shape}, dtype: {image.dtype}")

            # Skip if not 4D
            if len(image.shape) != 4:
                print(f"  Skipping - not a 4D image")
                continue

            # Process the 4D image (only first channel)
            projections = process_4d_microscopy_image(image, filename, process_first_channel_only=True)

            # Save each projection
            for proj_name, projection in projections.items():
                # Create output filename
                base_name = os.path.splitext(filename)[0]
                output_filename = f"{base_name}_{proj_name}.tif"
                output_path = os.path.join(output_dir, output_filename)

                # Convert to uint16 for saving
                if projection.dtype == np.float64 or projection.dtype == np.float32:
                    projection = img_as_uint(projection)

                # Save the projection
                tifffile.imwrite(output_path, projection)

                # Add to list of processed files
                processed_files.append(output_path)

            print(f"  Saved {len(projections)} projections")

        except Exception as e:
            print(f"  Error processing image {filename}: {e}")
            import traceback
            traceback.print_exc()
            continue

    return processed_files

# Main execution
if __name__ == "__main__":
    print("Starting 4D image processing with enhanced vacuole detection projection methods...")
    print(f"Input directory: {input_dir}")
    print(f"Output directory: {output_dir}")

    # Install required packages if necessary
    try:
        import pip
        print("Installing PyWavelets for wavelet decomposition projection...")
        !pip install PyWavelets
        print("PyWavelets installed successfully.")
    except Exception as e:
        print(f"Failed to install PyWavelets: {e}")
        print("Wavelet projection method will be skipped.")

    # Process all image files in the directory
    processed_files = process_all_images()

    # Display results
    if processed_files:
        print(f"\nSuccessfully processed {len(processed_files)} projections:")
        for output_file in processed_files[:5]:  # Show first 5
            print(f"  - {os.path.basename(output_file)}")
        if len(processed_files) > 5:
            print(f"  ... and {len(processed_files) - 5} more")

        # Try to display the first result in the notebook
        try:
            from IPython.display import Image
            print("\nDisplaying first result:")
            Image(processed_files[0])
        except Exception as e:
            print(f"Couldn't display image: {e}")
    else:
        print("No images were processed successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting 4D image processing with enhanced vacuole detection projection methods...
Input directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial
Output directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/trial24
Installing PyWavelets for wavelet decomposition projection...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 28.5 MB/s eta 0:00:00
PyWavelets installed successfully.
Found 1 image files. Processing each with multiple methods...
Processing image 1/1: denoised_1.4Pa_A1_19dec21_40x_L2RA_FlatA_seq006 (1).tif
  Image shape: (3, 21, 1024, 1024), dtype: float32
  Processing 4D image with shape (3, 21, 1024, 1024)
  Normalizing image values from range [-0.02250508777797222, 3.4323275089263916] to [0, 1]
  Applying inverse_weighted projection to channel 1...
  Successfully created channel_1_inverse

Traceback (most recent call last):
  File "<ipython-input-8-10d556433c1e>", line 977, in process_4d_microscopy_image
    proj = method_func(channel_data)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-8-10d556433c1e>", line 271, in local_minima_projection
    local_min = feature.peak_local_max(
                ^^^^^^^^^^^^^^^^^^^^^^^
TypeError: peak_local_max() got an unexpected keyword argument 'indices'


  Successfully created channel_1_vesicle_enhance projection
  Applying texture_discontinuity projection to channel 1...
  Successfully created channel_1_texture_discontinuity projection
  Applying adaptive_multi_threshold projection to channel 1...
  Successfully created channel_1_adaptive_multi_threshold projection
  Applying frequency_domain projection to channel 1...
  Successfully created channel_1_frequency_domain projection
  Applying topological_watershed projection to channel 1...
  Successfully created channel_1_topological_watershed projection
  Applying multi_scale_blob projection to channel 1...
  Successfully created channel_1_multi_scale_blob projection
  Applying adaptive_ensemble projection to channel 1...
  Applying texture_discontinuity_projection for ensemble...
  Applying adaptive_multi_threshold_projection for ensemble...
  Applying frequency_domain_projection for ensemble...
  Applying topological_watershed_projection for ensemble...
  Applying multi_scale_blob_de